# Visualização Orçamento SME

In [74]:
import pandas as pd
import requests
import json
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import glob
import datetime
from IPython.display import HTML
import pprint

pd.set_option('display.max_colwidth', 100)

In [75]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 50;

<IPython.core.display.Javascript object>

In [76]:
TOKEN = '198f959a5f39a1c441c7c863423264'
base_url = "https://gatewayapi.prodam.sp.gov.br:443/financas/orcamento/sof/v2.1.0"
headers = {'Authorization' : str('Bearer ' + TOKEN)}

# Carregando do arquivo

# Contratos

### Último ano

In [77]:
# json_files = glob.glob("wget-contratos/2017/*")

# frames = []

# for f in json_files:
#     e = json.load(open(f))
#     e_df = pd.DataFrame(e['lstContratos'])
#     frames.append(e_df)

# contratos_df = pd.concat(frames)

In [78]:
contratos_df = pd.read_json('dados/contratos-2017.json')

In [79]:
HTML(contratos_df.iloc[0:1].to_html())

,anoExercicio,codContrato,codEmpresa,codModalidade,codOrgao,codProcesso,codTipoContratacao,datAssinaturaContrato,datPublicacaoContrato,datVigencia,numOriginalContrato,txtDescricaoModalidade,txtDescricaoOrgao,txtObjetoContrato,txtRazaoSocial,txtTipoContratacao,valAditamentos,valAnulacao,valAnuladoEmpenho,valEmpenhadoLiquido,valLiquidado,valPago,valPrincipal,valReajustes,valTotalEmpenhado
0,NaN,18278,1,7,16,6016201700233541,5,None,None,10/09/2018,None,Inexigibilidade,Secretaria Municipal de Educação,"Contratação de Instrutor de libras IARA FAGUNDES DE SOUZA SILVA. RG nº 28.397.762-0, e inscrita...",PREFEITURA DO MUNICÍPIO DE SÃO PAULO,Nota de Empenho Fornecimento de Bens de Consumo e/ou Permanente,0.0,0.0,0.0,72834.0,51240.0,51240.0,72834.0,0.0,72834.0


# Empenhos

### Último ano

In [80]:
# json_files = glob.glob("wget-empenhos/2017/*")

# frames = []

# for f in json_files:
#     e = json.load(open(f))
#     e_df = pd.DataFrame(e['lstEmpenhos'])
#     frames.append(e_df)

# empenhos_df = pd.concat(frames)

In [81]:
empenhos_df = pd.read_json('dados/empenhos-2017.json')

### Todos anos

In [82]:
# json_files = glob.glob("wget-empenhos/*/*")

# frames = []

# for f in json_files:
#     e = json.load(open(f))
#     e_df = pd.DataFrame(e['lstEmpenhos'])
#     frames.append(e_df)

# empenhos_df_tot = pd.concat(frames)

## Baixar contratos e empenhos

In [83]:
# ver arquivo contratos_scraper.py
# e arquivo contratos_merge.py

## Arquivos já baixados

In [84]:
empenhos_contratos_df = pd.read_json('dados/empenhos-contratos-2017.json')

Quais colunas estão no empenhos_contratos e não no empenhos?

In [85]:
(empenhos_contratos_df.columns.values[
    np.invert(
        np.in1d(empenhos_contratos_df.columns.values, empenhos_df.columns.values)
        )
    ]
)

array(['codContrato', 'codProcesso'], dtype=object)

Exemplo de um contrato:

In [86]:
HTML(
    (contratos_df[contratos_df['codContrato'] == 25327])
.to_html())

,anoExercicio,codContrato,codEmpresa,codModalidade,codOrgao,codProcesso,codTipoContratacao,datAssinaturaContrato,datPublicacaoContrato,datVigencia,numOriginalContrato,txtDescricaoModalidade,txtDescricaoOrgao,txtObjetoContrato,txtRazaoSocial,txtTipoContratacao,valAditamentos,valAnulacao,valAnuladoEmpenho,valEmpenhadoLiquido,valLiquidado,valPago,valPrincipal,valReajustes,valTotalEmpenhado
6154,NaN,25327,1,12,16,6016201700541866,29,None,None,28/01/2018,None,Pregão,Secretaria Municipal de Educação,Aquisição de notebook,PREFEITURA DO MUNICÍPIO DE SÃO PAULO,Termo de Contrato para Aquisição de Bens de Consumo e/ou Permanente,0.0,0.0,0.0,40021740.0,40021740.0,40021740.0,40021740.0,0.0,40021740.0


Exemplo de um contrato e seu empenho relacionado:

In [87]:
HTML(
    (empenhos_contratos_df[empenhos_contratos_df['codContrato'] == 25327])
.to_html())

,anoEmpenho,codCategoria,codContrato,codElemento,codEmpenho,codEmpresa,codFonteRecurso,codFuncao,codGrupo,codItemDespesa,codModalidade,codOrgao,codProcesso,codPrograma,codProjetoAtividade,codSubElemento,codSubFuncao,codUnidade,datEmpenho,mesEmpenho,nomEmpresa,numCpfCnpj,numReserva,txtCategoriaEconomica,txtDescricaoElemento,txtDescricaoFonteRecurso,txtDescricaoFuncao,txtDescricaoItemDespesa,txtDescricaoOrgao,txtDescricaoPrograma,txtDescricaoProjetoAtividade,txtDescricaoSubElemento,txtDescricaoSubFuncao,txtDescricaoUnidade,txtGrupoDespesa,txtModalidadeAplicacao,txtRazaoSocial,valAnuladoEmpenho,valEmpenhadoLiquido,valLiquidado,valPagoExercicio,valPagoRestos,valTotalEmpenhado
12534,2017,4,25327,52,120981,1,0,12,4,99,90,16,6016201700541866,3011,2818,6,126,10,08/12/2017,12,PREFEITURA DO MUNICÍPIO DE SÃO PAULO,81243735001977,69581,Despesas de Capital,Equipamentos e Material Permanente,Tesouro Municipal,Educação,Outros Aparelhos e Equipamentos de Comunicação,Secretaria Municipal de Educação,Modernização e descentralização da gestão e prestação de serviços,"Aquisição de Materiais, Equipamentos e Serviços de Informação e Comuni",Aparelhos e Equipamentos de Comunicação,Tecnologia da Informação,Gabinete do Secretário,Investimentos,Aplicações Diretas,POSITIVO TECNOLOGIA S.A.,0.0,40021740.0,0.0,0.0,40021740.0,40021740.0


Número de empenhos sem contratos relacionados:

In [88]:
len(empenhos_df) - len(empenhos_contratos_df)

41924

## Join contratos e empenhos

Quais colunas estão no contratos e não no empenhos_contratos?

In [89]:
list(contratos_df.columns.values[
    np.invert(
        np.in1d(contratos_df.columns.values, empenhos_contratos_df.columns.values)
        )
    ]
)

['anoExercicio',
 'codTipoContratacao',
 'datAssinaturaContrato',
 'datPublicacaoContrato',
 'datVigencia',
 'numOriginalContrato',
 'txtDescricaoModalidade',
 'txtObjetoContrato',
 'txtTipoContratacao',
 'valAditamentos',
 'valAnulacao',
 'valPago',
 'valPrincipal',
 'valReajustes']

In [90]:
# merge
delta = list(contratos_df.columns.values[
    np.invert(
        np.in1d(contratos_df.columns.values, empenhos_contratos_df.columns.values)
        )
    ]
)
delta.append('codContrato')
contratos_df_delta = contratos_df[delta]
temp = pd.merge(empenhos_contratos_df[['codEmpenho', 'codContrato']], contratos_df_delta, how='inner', 
         on='codContrato')
empenhos_df = pd.merge(empenhos_df, temp, how='left', 
         on='codEmpenho')

In [91]:
empenhos_df.columns.values

array(['anoEmpenho', 'cat_1', 'codCategoria', 'codElemento', 'codEmpenho',
       'codEmpresa', 'codFonteRecurso', 'codFuncao', 'codGrupo',
       'codItemDespesa', 'codModalidade', 'codOrgao', 'codPrograma',
       'codProjetoAtividade', 'codSubElemento', 'codSubFuncao',
       'codUnidade', 'datEmpenho', 'mesEmpenho', 'nomEmpresa',
       'numCpfCnpj', 'numReserva', 'txtCategoriaEconomica',
       'txtDescricaoElemento', 'txtDescricaoFonteRecurso',
       'txtDescricaoFuncao', 'txtDescricaoItemDespesa',
       'txtDescricaoOrgao', 'txtDescricaoPrograma',
       'txtDescricaoProjetoAtividade', 'txtDescricaoSubElemento',
       'txtDescricaoSubFuncao', 'txtDescricaoUnidade', 'txtGrupoDespesa',
       'txtModalidadeAplicacao', 'txtRazaoSocial', 'valAnuladoEmpenho',
       'valEmpenhadoLiquido', 'valLiquidado', 'valPagoExercicio',
       'valPagoRestos', 'valTotalEmpenhado', 'codContrato',
       'anoExercicio', 'codTipoContratacao', 'datAssinaturaContrato',
       'datPublicacaoContrato

##  Índice para corrigir inflação

In [92]:
inflacao_df = pd.DataFrame(columns=["ano", "indice"], 
  data=[
    [2006, 1.8974051],
    [2007, 1.7862712],
    [2008, 1.5965463],
    [2009, 1.6222454],
    [2010, 1.4711348],
    [2011, 1.3885172],
    [2012, 1.2982224],
    [2013, 1.229265],
    [2014, 1.1859287],
    [2015, 1.0714226],
    [2016, 1.0000453],
    [2017, 1]
  ])

## Helper para explorar grupos

In [93]:
def groupSum(df):
    terms = [
        'txtCategoriaEconomica',
        'txtDescricaoElemento',
        'txtDescricaoFonteRecurso',
        # 'txtDescricaoFuncao',
        'txtDescricaoItemDespesa',
        # 'txtDescricaoOrgao',
        'txtDescricaoPrograma',
        'txtDescricaoProjetoAtividade',
        'txtDescricaoSubElemento',
        'txtDescricaoSubFuncao',
        'txtDescricaoUnidade',
        'txtGrupoDespesa',
        'txtModalidadeAplicacao',
        'txtRazaoSocial'
    ]
    for el in terms:
        result = (df
         .groupby(el)['valTotalEmpenhado', 'valLiquidado']
         .sum().apply(lambda x: x / 1000000).sort_values('valLiquidado', ascending=False)
        )
        display(result)
    display(
    # HTML(
    df[['txtObjetoContrato']].drop_duplicates(subset='txtObjetoContrato')
        # .to_html())
    )

### Helper para contratos

In [94]:
def groupSumContratos(df):
    terms = ['anoExercicio',
        'txtDescricaoModalidade',
        'txtDescricaoOrgao',
        'txtObjetoContrato',
        'txtRazaoSocial',
        'txtTipoContratacao',
    ]
    for el in terms:
        result = (df
         .groupby(el)['valTotalEmpenhado', 'valLiquidado']
         .sum().apply(lambda x: x / 1000000).sort_values('valLiquidado', ascending=False)
        )
        display(result)

# Atenção!

Os valores estão em milhões de reais, a não ser que indicado o contrário.

## Explorando dados

### Agrupamentos

In [95]:
groupSum(empenhos_df)

,valTotalEmpenhado,valLiquidado
txtCategoriaEconomica,,
Despesas Correntes,12798.904536,10172.665375
Despesas de Capital,167.358136,63.463741


,valTotalEmpenhado,valLiquidado
txtDescricaoElemento,,
Vencimentos e Vantagens Fixas - Pessoal Civil,7227.363117,5770.231471
Outros Serviços de Terceiros - Pessoa Jurídica,2370.502024,2073.220587
Outros Serviços de Terceiros -Pessoa Jurídica,1902.575828,1367.463481
Auxílio-Alimentação,434.820030,430.542262
Material de Consumo,273.308796,202.342032
"Material, Bem ou Serviço para Distribuição Gratuita",390.772614,162.051071
Outros Serviços de Terceiros - Pessoa Física,140.887005,117.782310
Obras e Instalações,89.889729,39.131728
Contribuições,23.877403,19.871110


,valTotalEmpenhado,valLiquidado
txtDescricaoFonteRecurso,,
Tesouro Municipal,7533.654836,5788.261271
Fundo Constitucional da Educação,4440.279767,3827.119838
Transferências Federais,938.337874,587.762549
Outras Fontes,33.979873,16.566437
Transferências Estaduais,20.010323,16.419021


,valTotalEmpenhado,valLiquidado
txtDescricaoItemDespesa,,
RPPS - PMSP,7224.500741,5768.052052
Convênio - Creche,2230.700574,1955.371328
Limpeza de Ambientes,460.588315,404.674394
Refeições Prontas,593.334812,328.169695
Auxílio-Refeição - RPPS,231.914947,231.900788
Vale-Alimentação - RPPS,201.652925,197.393040
Transporte Escolar - EMEF,226.955018,190.285810
"Guarda, Vigilância e Segurança em Geral",214.285475,180.449088
Diversos Gêneros Alimentícios,159.092806,118.280276


,valTotalEmpenhado,valLiquidado
txtDescricaoPrograma,,
Melhoria da qualidade e ampliação do acesso à educação,12278.342078,9737.146846
Suporte Administrativo,487.094474,385.216074
Direitos da pessoa com deficiência,71.587204,56.705663
Modernização e descentralização da gestão e prestação de serviços,113.625732,44.050750
Superação da extrema pobreza,11.955134,10.704482
Promoção da cidade como centro de eventos e destino turístico de refer,2.056456,1.448251
"Participação, transparência e controle social da administração pública",1.601595,0.857049


...

,valTotalEmpenhado,valLiquidado
txtDescricaoUnidade,,
Gabinete do Secretário,9375.871273,7365.919168
Departamento de Alimentação Escolar,864.352108,555.403268
Diretoria Regional de Educação - Campo Limpo,259.667430,236.657658
Diretoria Regional de Educação - Guaianases,296.241506,214.297709
Diretoria Regional de Educação - São Miguel,222.969903,210.591931
Diretoria Regional de Educação - Pirituba,202.860499,191.569627
Diretoria Regional de Educação - Ipiranga,188.018721,176.640245
Diretoria Regional de Educação - Freguesia/Brasilândia,212.657771,175.734458
Diretoria Regional de Educação - Itaquera,207.890332,172.945943


,valTotalEmpenhado,valLiquidado
txtGrupoDespesa,,
Pessoal e Encargos Sociais,7227.696087,5770.247369
Outras Despesas Correntes,5571.208449,4402.418006
Investimentos,167.358136,63.463741


,valTotalEmpenhado,valLiquidado
txtModalidadeAplicacao,,
Aplicações Diretas,10564.154302,8136.470299
Transferências a Instituições Privadas sem Fins Lucrativos,2401.965902,2099.546763
"Apl.Dir.Dec.Op. entre Órgãos,Fundos e Ent.Int.Orç.Fiscal e Seg.Social",0.142468,0.112054


,valTotalEmpenhado,valLiquidado
txtRazaoSocial,,
FOLHA DE PAGAMENTO _ EXECUTIVO,7224.500741,5768.052052
SERV.SEC.MUN. DE EDUCAÇÃO - AUX. REFEIÇÃO,434.818454,430.542262
SHA COMERCIO DE ALIMENTOS LTDA,190.741737,118.178282
CIA DE SANEAMENTO BÁSICO DO ESTADO DE SÃO PAULO SABESP,126.612305,93.082309
DENJUD REFEIÇÕES COLETIVAS ADMINISTRAÇÃO E SERVIÇOS LTDA,144.606175,77.517622
COMERCIAL MILANO BRASIL LTDA,92.039220,74.653773
APETECE SISTEMAS DE ALIMENTACAO S/A,161.392758,73.424133
TANGARÁ IMP.E EXP.S/A,61.845260,55.286151
ELETROPAULO METROPOLITANA ELETRICIDADE DE SÃO PAULO S.A.,66.003813,44.973127


,txtObjetoContrato
0,IT - 1º REPASSE PTRF PARA APM DA EMEI CAP. ALBERTO MENDES JUNIOR - DE ACORDO COM A LEI Nº 9394...
1,MP - 1º REPASSE PTRF PARA APM DO EMEI CLEMENCIA FERREIRA DA SILVA - DE ACORDO COM A LEI Nº 9394/...
2,"IT - 1º REPASSE PTRF PARA APM DO CEI JARDIM NAZARÉ - DE ACORDO COM A LEI Nº 9394//96, LEI MUNICI..."
3,"IT - 1º REPASSE PTRF PARA APM DA EMEI OVIDIO DECROLY - DE ACORDO COM A LEI Nº 9394//96, LEI MUN..."
4,MP - 1º REPASSE PTRF PARA APM DO EMEI PROFª MARIA DA LUZ SILVA DE OLIVEIRA - DE ACORDO COM A LE...
5,MP - 1º REPASSE PTRF PARA APM DO EMEI PROFª CELIA RIBEIRO LANDIM - DE ACORDO COM A LEI Nº 9394//...
6,IT - 1º REPASSE PTRF PARA APM DO CEI CONJ. HABITACIONAL TEXIMA - DE ACORDO COM A LEI Nº 9394//96...
7,MP - 1º REPASSE PTRF PARA APM DO EMEI DR. ANTONIO ROBERTO ALVES BRAGA - DE ACORDO COM A LEI Nº ...
8,"IT - 1º REPASSE PARA APM DA EMEF EUZEBIO ROCHA FILHO - DE ACORDO COM A LEI Nº 9394//96, LEI MUNI..."
9,"IT - 1º REPASSE PTRF PARA APM DO CEI VER. LIBERO ANCONA LOPES - DE ACORDO COM A LEI Nº 9394//96,..."


## Contratos

In [96]:
groupSumContratos(contratos_df)

,valTotalEmpenhado,valLiquidado
anoExercicio,,


,valTotalEmpenhado,valLiquidado
txtDescricaoModalidade,,
Pregão,943.559736,641.162346
Não Aplicável,610.937541,474.659843
Dispensa de Licitação,591.269262,427.488146
Inexigibilidade,61.632085,52.071161
Leilão,0.209999,0.209999
Tomada de Preços,0.677581,0.143517
Concorrência,0.115772,0.115772
Convite,0.000000,0.000000


,valTotalEmpenhado,valLiquidado
txtDescricaoOrgao,,
Secretaria Municipal de Educação,2208.401976,1595.850786


,valTotalEmpenhado,valLiquidado
txtObjetoContrato,,
Aquisição de Kit de Uniforme Escolar,61.576593,61.576593
"Serviços de Nutrição e Alimentação Escolar - Regiões de GUAIANASES, ITAQUERA e PENHA \n\n",50.274381,40.530609
Aquisição de notebook,40.021740,40.021740
Serviços de Nutrição e Alimentação Escolar - Região da FREGUESIA DO Ó e SÃO MATEUS,44.781926,39.143829
Aquisição de kits de uniforme escolar lotes 04 e 06,117.814656,35.933590
Serviços de Nutrição e Alimentação Escolar - Regiões - IPIRANGA E SÃO MIGUEL,42.442385,32.896587
Limpeza de CEUs.,34.210382,32.482408
Aquisição de Kit de Uniforme Escolar,29.558169,29.558169
Aquisição de kits de uniforme escolar lote 02,26.810556,26.810556


,valTotalEmpenhado,valLiquidado
txtRazaoSocial,,
PREFEITURA DO MUNICÍPIO DE SÃO PAULO,2208.401976,1595.850786


,valTotalEmpenhado,valLiquidado
txtTipoContratacao,,
Termo de Contrato - Serviços,1083.524679,734.338001
Nota de Empenho Fornecimento de Bens de Consumo e/ou Permanente,396.009759,280.776631
Termo de Colaboração - Lei Federal 13.019/2014,360.355316,249.850884
Termo de Contrato para Aquisição de Bens de Consumo e/ou Permanente,176.563671,169.319828
Termo de Convênio,103.758882,87.287964
Termo de Compromisso PTRF,40.178396,38.926438
Termo de Repasse PNAE,23.863291,19.722606
Acordo de Cooperação - Lei Federal 13.019/2014,10.918979,7.054200
Termo de Fomento - Lei Federal 13.019/2014,8.469562,5.503498


In [97]:
HTML(
    (contratos_df[['codContrato', 'txtObjetoContrato', 'valLiquidado']])
    .sort_values('valLiquidado', ascending=False)
.to_html())

,codContrato,txtObjetoContrato,valLiquidado
4672,1609,"Serviços de Nutrição e Alimentação Escolar - Regiões de GUAIANASES, ITAQUERA e PENHA \n\n",40530608.51
6154,25327,Aquisição de notebook,40021740.00
4251,1594,Serviços de Nutrição e Alimentação Escolar - Região da FREGUESIA DO Ó e SÃO MATEUS,39143829.10
5985,25275,Aquisição de Kit de Uniforme Escolar,35761763.20
4684,423,Aquisição de kits de uniforme escolar lotes 04 e 06,34755365.19
4641,1615,Serviços de Nutrição e Alimentação Escolar - Regiões - IPIRANGA E SÃO MIGUEL,32896586.93
5073,4480,"Serviços de vigilância CEUs - lotes 01, 02, 04 e 05",25222766.76
6082,25283,Aquisição de Kit de Uniforme Escolar,24433320.60
3874,12542,Contratação de serviços de nutrição e alimentação escolar- Lote 4 - CAPELA DO SOCORRO.,23420834.83
5199,2767,AQUISIÇÃO DE KITS DE UNIFORME ESCOLAR - LOTES 04 E 06,22973737.60


In [98]:
cod = 1609
filter_df = empenhos_contratos_df[empenhos_contratos_df['codContrato'] == cod]
HTML(filter_df.to_html())
# filter_df.txtDescricaoProjetoAtividade

,anoEmpenho,codCategoria,codContrato,codElemento,codEmpenho,codEmpresa,codFonteRecurso,codFuncao,codGrupo,codItemDespesa,codModalidade,codOrgao,codProcesso,codPrograma,codProjetoAtividade,codSubElemento,codSubFuncao,codUnidade,datEmpenho,mesEmpenho,nomEmpresa,numCpfCnpj,numReserva,txtCategoriaEconomica,txtDescricaoElemento,txtDescricaoFonteRecurso,txtDescricaoFuncao,txtDescricaoItemDespesa,txtDescricaoOrgao,txtDescricaoPrograma,txtDescricaoProjetoAtividade,txtDescricaoSubElemento,txtDescricaoSubFuncao,txtDescricaoUnidade,txtGrupoDespesa,txtModalidadeAplicacao,txtRazaoSocial,valAnuladoEmpenho,valEmpenhadoLiquido,valLiquidado,valPagoExercicio,valPagoRestos,valTotalEmpenhado
14837,2017,3,1609,39,10383,1,2,12,3,99,90,16,6016201700010780,3010,6553,41,306,24,01/02/2017,12,PREFEITURA DO MUNICÍPIO DE SÃO PAULO,60166832000104,7929,Despesas Correntes,Outros Serviços de Terceiros -Pessoa Jurídica,Transferências Federais,Educação,Refeições Prontas,Secretaria Municipal de Educação,Melhoria da qualidade e ampliação do acesso à educação,Alimentação Escolar,Fornecimento de Alimentação,Alimentação e Nutrição,Departamento de Alimentação Escolar,Outras Despesas Correntes,Aplicações Diretas,APETECE SISTEMAS DE ALIMENTACAO S/A,2229482.30,15930898.36,15930898.36,15930898.36,0.0,18160380.66
14838,2017,3,1609,39,10424,1,2,12,3,99,90,16,6016201700010780,3010,6553,41,306,24,01/02/2017,12,PREFEITURA DO MUNICÍPIO DE SÃO PAULO,60166832000104,7929,Despesas Correntes,Outros Serviços de Terceiros -Pessoa Jurídica,Transferências Federais,Educação,Refeições Prontas,Secretaria Municipal de Educação,Melhoria da qualidade e ampliação do acesso à educação,Alimentação Escolar,Fornecimento de Alimentação,Alimentação e Nutrição,Departamento de Alimentação Escolar,Outras Despesas Correntes,Aplicações Diretas,APETECE SISTEMAS DE ALIMENTACAO S/A,1430733.10,12363102.26,12363102.26,12363102.26,0.0,13793835.36
14839,2017,3,1609,39,10438,1,2,12,3,99,90,16,6016201700010780,3010,6553,41,306,24,01/02/2017,12,PREFEITURA DO MUNICÍPIO DE SÃO PAULO,60166832000104,7929,Despesas Correntes,Outros Serviços de Terceiros -Pessoa Jurídica,Transferências Federais,Educação,Refeições Prontas,Secretaria Municipal de Educação,Melhoria da qualidade e ampliação do acesso à educação,Alimentação Escolar,Fornecimento de Alimentação,Alimentação e Nutrição,Departamento de Alimentação Escolar,Outras Despesas Correntes,Aplicações Diretas,APETECE SISTEMAS DE ALIMENTACAO S/A,6083556.73,12236607.89,12236607.89,12236607.89,0.0,18320164.62


### Processando termos

In [99]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk

txt = contratos_df['txtObjetoContrato']
stopwords = nltk.corpus.stopwords.words('portuguese')

v = TfidfVectorizer(stop_words=stopwords)
x = v.fit(txt)
vocabulary = x.vocabulary_
vocabulary
[(k, vocabulary[k]) for k in sorted(vocabulary, key=vocabulary.get, reverse=True)]

[('ônibus', 5847),
 ('ômega', 5846),
 ('óleo', 5845),
 ('ítem', 5844),
 ('étnico', 5843),
 ('âmbito', 5842),
 ('ávore', 5841),
 ('ávila', 5840),
 ('áurea', 5839),
 ('ás', 5838),
 ('árvores', 5837),
 ('árvore', 5836),
 ('área', 5835),
 ('árbitros', 5834),
 ('álvaro', 5833),
 ('álcool', 5832),
 ('álamos', 5831),
 ('águia', 5830),
 ('águas', 5829),
 ('água', 5828),
 ('ágape', 5827),
 ('àgua', 5826),
 ('àgo', 5825),
 ('zéglio', 5824),
 ('zuleika', 5823),
 ('zuando', 5822),
 ('zito', 5821),
 ('zita', 5820),
 ('zezinho', 5819),
 ('zello', 5818),
 ('zelia', 5817),
 ('zeglio', 5816),
 ('zefinha', 5815),
 ('zefa', 5814),
 ('zeca', 5813),
 ('zazá', 5812),
 ('zampieri', 5811),
 ('zacaria', 5810),
 ('yvone', 5809),
 ('yvette', 5808),
 ('yuri', 5807),
 ('yoshio', 5806),
 ('yolanda', 5805),
 ('yamaguchi', 5804),
 ('yahwe', 5803),
 ('xxiii', 5802),
 ('xxi', 5801),
 ('xv', 5800),
 ('xlevy', 5799),
 ('xis', 5798),
 ('xinha', 5797),
 ('xiii', 5796),
 ('xi', 5795),
 ('xg', 5794),
 ('xavier', 5793),
 ('xa

In [100]:
teste = (contratos_df[
    (contratos_df['txtObjetoContrato'].str.contains('vigil', case=False))
])
groupSumContratos(teste)

,valTotalEmpenhado,valLiquidado
anoExercicio,,


,valTotalEmpenhado,valLiquidado
txtDescricaoModalidade,,
Pregão,60.991713,42.347136
Dispensa de Licitação,1.707695,1.455421


,valTotalEmpenhado,valLiquidado
txtDescricaoOrgao,,
Secretaria Municipal de Educação,62.699409,43.802557


,valTotalEmpenhado,valLiquidado
txtObjetoContrato,,
"Serviços de vigilância CEUs - lotes 01, 02, 04 e 05",36.855961,25.222767
"Serviços de vigilância CEUs - lotes 03, 06 e 07",23.945679,16.985714
SP/MP CONTRATAÇÃO DE SERVIÇOS DE SEGURANÇA E VIGILÂNCIA DESARMADA PARA A DRE-MP SENDO 5 POSTOS DIURNOS E 3 POSTOS NOTURNO PERÍODO DE 01/03/2017 À 28/02/2018.,1.245441,1.158513
CONTRATAÇÃO DE EMPRESA ESPECIALIZADA EM VIGILANCIA DESARMADA PARA ATENDIMENTO DA SEDE DA DRE,0.172787,0.172787
PREGÃO ELETRÔNICO 12/2017 - CONTRATAÇÃO DE EMPRESA PARA PRESTAÇ~]AO DE SERVIÇOS DE SEGURANÇA E VIGILÂNCIA PARA O ALMOXARIFADO,0.190073,0.138655
"CONTRATAÇÃ ODE EMPRESA ESPECIALIZADA NA PRESTAÇÃ ODE SERVIÇO DE VIGILANCIA E SEGURANÇA PATRIMONIAL DESARMADA, COM INSTALAÇÃO E MANUTENÇÃO CORRETIVA PREVENTIVA DE EQUIPAMENTO ELETRONICO.",0.289467,0.124121


,valTotalEmpenhado,valLiquidado
txtRazaoSocial,,
PREFEITURA DO MUNICÍPIO DE SÃO PAULO,62.699409,43.802557


,valTotalEmpenhado,valLiquidado
txtTipoContratacao,,
Termo de Contrato - Serviços,60.801640,42.208481
Nota de Empenho Fornecimento de Bens de Consumo e/ou Permanente,1.897768,1.594076


# Orçamento - Itens

* Fontes dos recursos (para todos itens)

## 1. Pessoal
1. Docentes
2. Getores
3. Pessoal de apoio
4. Aposentadorias

## 2. Transporte Escolar
1. Passe Livre
2. TEG
3. Outros?

## 3. Funcionamento das escolas
1. Limpeza
2. Vigilância
3. Manutenção
4. Outros?

## 4. Uniformes e materiais escolares
1. Uniformes
2. Materiais (escolares, esportivos, educativos, didáticos, artísticos... como subdividir?)
3. Outros?

## 5. Parcerias
1. Convênios creches
2. Educação especial (como subdividir?)
3. MOVA
4. Outros?

## 6. Obras e reformas
1. Novas construções (possível subdividir em unidades educacionais e outros?)
2. Reformas (idem?)
3. Outros

## 7. Alimentação
1. Refeições prontas
2. Leve-Leite
3. Preparo e distribuição
4. Insumos (para refeições preparadas pela própria SME? Como subdividir?)
5. Outros?

## 8. Pedagógico e formação
1. Cursos e formação
2. Elaboração de materiais didáticos
3. Outros?

## 9. Tecnologia da informação
1. Internet
2. Manutenção de equipamentos
3. Outros?

## 1. Pessoal

### Filtro e exploração geral

#### Checar: filtro está correto? Está realmente capturando todas despesas com pessoal?

In [101]:
pessoal_filter = (
    (empenhos_df['txtDescricaoItemDespesa'].str.contains('rpps', case=False))
    |
    (empenhos_df['txtGrupoDespesa'].str.contains('pessoal', case=False))
    |
    (empenhos_df['txtDescricaoItemDespesa'].str.contains('Estagiários', case=False))
)
pessoal = (empenhos_df[pessoal_filter])
empenhos_df.loc[pessoal_filter, 'cat_1'] = 'pessoal'
groupSum(pessoal)

,valTotalEmpenhado,valLiquidado
txtCategoriaEconomica,,
Despesas Correntes,7721.646989,6250.746595


,valTotalEmpenhado,valLiquidado
txtDescricaoElemento,,
Vencimentos e Vantagens Fixas - Pessoal Civil,7227.363117,5770.231471
Auxílio-Alimentação,433.567871,429.293828
Outros Serviços de Terceiros - Pessoa Jurídica,34.734801,27.796770
Auxílio-Transporte,17.757254,17.757254
Outros Auxílios Financeiros a Pessoas Físicas,7.890976,5.651375
Ressarcimento de Despesas de Pessoal Requisitado,0.332970,0.015898


,valTotalEmpenhado,valLiquidado
txtDescricaoFonteRecurso,,
Fundo Constitucional da Educação,4440.279767,3827.119838
Tesouro Municipal,3278.815338,2421.606567
Transferências Federais,2.551884,2.020191


,valTotalEmpenhado,valLiquidado
txtDescricaoItemDespesa,,
RPPS - PMSP,7224.500741,5768.052052
Auxílio-Refeição - RPPS,231.914947,231.900788
Vale-Alimentação - RPPS,201.652925,197.393040
Concessão de Bolsa-Auxílio para Estagiários,34.734801,27.796770
Auxílio-Transporte - RPPS,17.757254,17.757254
Concessão de auxílio-transporte a estagiários,7.890976,5.651375
Outras Despesas Fixas - Pessoal Civil,2.862376,2.179419
PMSP,0.332970,0.015898


,valTotalEmpenhado,valLiquidado
txtDescricaoPrograma,,
Melhoria da qualidade e ampliação do acesso à educação,7271.343489,5886.305670
Suporte Administrativo,426.666089,344.566703
Direitos da pessoa com deficiência,23.637411,19.874223


...

,valTotalEmpenhado,valLiquidado
txtDescricaoUnidade,,
Gabinete do Secretário,7721.646989,6250.746595


,valTotalEmpenhado,valLiquidado
txtGrupoDespesa,,
Pessoal e Encargos Sociais,7227.696087,5770.247369
Outras Despesas Correntes,493.950901,480.499227


,valTotalEmpenhado,valLiquidado
txtModalidadeAplicacao,,
Aplicações Diretas,7679.021212,6217.298451
Transferências a Instituições Privadas sem Fins Lucrativos,42.625777,33.448145


,valTotalEmpenhado,valLiquidado
txtRazaoSocial,,
FOLHA DE PAGAMENTO _ EXECUTIVO,7224.500741,5768.052052
SERV.SEC.MUN. DE EDUCAÇÃO - AUX. REFEIÇÃO,433.567871,429.293828
CENTRO DE INTEGRAÇÃO EMPRESA ESCOLA CIEE,42.625777,33.448145
SERVIDORES DA SECRETARIA MUNICIPAL DE EDUCAÇÃO - AUX. TRANSPORTE,17.757254,17.757254
CARLOS AUGUSTO DE OLIVEIRA GONÇALVES,0.026935,0.026935
ROSELICE APARECIDA CARDOSO SANTOS,0.026918,0.026918
LUCIENE SOARES MARCOLINO,0.026935,0.026831
SANDRA MARIA AMANCIO DUARTE,0.026779,0.026779
ANDREA REGINA DE CARVALHO CASANOVA,0.026733,0.026733


,txtObjetoContrato
5218,NaN


## 1.1. Docentes

#### Checar: será que realmente esse filtro compreende todos docentes? Me parece sub representado.

In [102]:
docentes_filter = pessoal_filter & (
    (empenhos_df['txtDescricaoSubFuncao'].str.contains('Educação Básica', case=False))
    &
    ~(empenhos_df['txtDescricaoItemDespesa'].str.contains('Estagiários', case=False))
)
docentes = (pessoal[docentes_filter])
empenhos_df.loc[docentes_filter, 'cat_2'] = 'docentes'
display(len(empenhos_df[empenhos_df['cat_2'] == 'docentes']) == len(docentes))
groupSum(docentes)

/home/bpl/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


True

,valTotalEmpenhado,valLiquidado
txtCategoriaEconomica,,
Despesas Correntes,7252.825865,5872.021815


,valTotalEmpenhado,valLiquidado
txtDescricaoElemento,,
Vencimentos e Vantagens Fixas - Pessoal Civil,6817.728694,5441.198688
Auxílio-Alimentação,417.984505,413.710462
Auxílio-Transporte,17.112666,17.112666


,valTotalEmpenhado,valLiquidado
txtDescricaoFonteRecurso,,
Fundo Constitucional da Educação,4440.279767,3827.119838
Tesouro Municipal,2812.546098,2044.901978


...

,valTotalEmpenhado,valLiquidado
txtDescricaoUnidade,,
Gabinete do Secretário,7252.825865,5872.021815


,valTotalEmpenhado,valLiquidado
txtGrupoDespesa,,
Pessoal e Encargos Sociais,6817.728694,5441.198688
Outras Despesas Correntes,435.097171,430.823128


,valTotalEmpenhado,valLiquidado
txtModalidadeAplicacao,,
Aplicações Diretas,7252.825865,5872.021815


,valTotalEmpenhado,valLiquidado
txtRazaoSocial,,
FOLHA DE PAGAMENTO _ EXECUTIVO,6817.728694,5441.198688
SERV.SEC.MUN. DE EDUCAÇÃO - AUX. REFEIÇÃO,417.984505,413.710462
SERVIDORES DA SECRETARIA MUNICIPAL DE EDUCAÇÃO - AUX. TRANSPORTE,17.112666,17.112666


,txtObjetoContrato
5218,NaN


## 1.2. Gestores

#### Checar: será que realmente esse filtro compreende todos gestores? Depende do filtro anterior estar certo

In [103]:
gestores_filter = pessoal_filter & (
    (pessoal['txtDescricaoSubFuncao'].str.contains('Administração Geral', case=False))
    &
    ~(pessoal['txtDescricaoItemDespesa'].str.contains('Estagiários', case=False))
)
gestores = (pessoal[gestores_filter])
empenhos_df.loc[gestores_filter, 'cat_2'] = 'gestores'
display(len(empenhos_df[empenhos_df['cat_2'] == 'gestores']) == len(gestores))
groupSum(gestores)

/home/bpl/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


True

,valTotalEmpenhado,valLiquidado
txtCategoriaEconomica,,
Despesas Correntes,423.332971,343.097216


,valTotalEmpenhado,valLiquidado
txtDescricaoElemento,,
Vencimentos e Vantagens Fixas - Pessoal Civil,406.772047,326.853364
Auxílio-Alimentação,15.583366,15.583366
Auxílio-Transporte,0.644588,0.644588
Ressarcimento de Despesas de Pessoal Requisitado,0.332970,0.015898


,valTotalEmpenhado,valLiquidado
txtDescricaoFonteRecurso,,
Tesouro Municipal,423.332971,343.097216


...

,valTotalEmpenhado,valLiquidado
txtDescricaoUnidade,,
Gabinete do Secretário,423.332971,343.097216


,valTotalEmpenhado,valLiquidado
txtGrupoDespesa,,
Pessoal e Encargos Sociais,407.105017,326.869262
Outras Despesas Correntes,16.227954,16.227954


,valTotalEmpenhado,valLiquidado
txtModalidadeAplicacao,,
Aplicações Diretas,423.332971,343.097216


,valTotalEmpenhado,valLiquidado
txtRazaoSocial,,
FOLHA DE PAGAMENTO _ EXECUTIVO,406.772047,326.853364
SERV.SEC.MUN. DE EDUCAÇÃO - AUX. REFEIÇÃO,15.583366,15.583366
SERVIDORES DA SECRETARIA MUNICIPAL DE EDUCAÇÃO - AUX. TRANSPORTE,0.644588,0.644588
MINISTÉRIO DO PLANEJAMENTO ORÇAMENTO E GESTÃO - MP,0.332970,0.015898


,txtObjetoContrato
6256,NaN


## 1.3. Pessoal de apoio

#### Como filtrar?

## 1.4. Aposentadorias

#### Como filtrar?

## 1.5 Adiantamentos (?)

In [104]:
adiantamentos = (empenhos_df[
    (empenhos_df['txtDescricaoItemDespesa'].str.contains('Adiantamento', case=False))
])
groupSum(adiantamentos)

,valTotalEmpenhado,valLiquidado
txtCategoriaEconomica,,
Despesas Correntes,12.447244,12.213167


,valTotalEmpenhado,valLiquidado
txtDescricaoElemento,,
Outros Serviços de Terceiros -Pessoa Jurídica,12.435564,12.204974
Diárias - Civil,0.011680,0.008193


,valTotalEmpenhado,valLiquidado
txtDescricaoFonteRecurso,,
Tesouro Municipal,12.447244,12.213167


,valTotalEmpenhado,valLiquidado
txtDescricaoItemDespesa,,
"Adiantamento - Incisos I, II e III, Art. 2º da Lei nº 10.513/88",12.433464,12.202874
"Adiantamento - Inciso VI, Art. 2º da Lei 10.513/88",0.008112,0.004625
"Adiantamento - Inciso VI, Art. 2º da Lei nº 10.513/88",0.003568,0.003568
"Adiantamento - Inciso VII, Art. 2º da Lei nº 10.513/88",0.002100,0.002100


,valTotalEmpenhado,valLiquidado
txtDescricaoPrograma,,
Melhoria da qualidade e ampliação do acesso à educação,12.063909,11.840519
Suporte Administrativo,0.381235,0.370548
"Participação, transparência e controle social da administração pública",0.002100,0.002100


...

,valTotalEmpenhado,valLiquidado
txtDescricaoUnidade,,
Diretoria Regional de Educação - São Miguel,1.542215,1.527715
Diretoria Regional de Educação - Pirituba,1.365650,1.321850
Diretoria Regional de Educação - Campo Limpo,1.311960,1.294360
Diretoria Regional de Educação - São Mateus,1.230525,1.213925
Diretoria Regional de Educação - Guaianases,1.213650,1.202350
Diretoria Regional de Educação - Penha,1.019648,0.985852
Diretoria Regional de Educação - Butantã,0.985692,0.969792
Diretoria Regional de Educação - Ipiranga,0.828100,0.799700
Diretoria Regional de Educação - Freguesia/Brasilândia,0.729700,0.720700


,valTotalEmpenhado,valLiquidado
txtGrupoDespesa,,
Outras Despesas Correntes,12.447244,12.213167


,valTotalEmpenhado,valLiquidado
txtModalidadeAplicacao,,
Aplicações Diretas,12.447244,12.213167


,valTotalEmpenhado,valLiquidado
txtRazaoSocial,,
ANDREA CALDEO CALVO,0.040000,0.040000
JOSÉ VALDESIO DE SOUSA,0.040000,0.040000
ANGELINA LETTIERI CARVALHO,0.039000,0.039000
SILVIA BUENO PAQUES N.DE OLIVEIRA,0.035000,0.035000
EDSON NOVAES PRADO,0.034000,0.034000
IRENE CRAVO CAMPOS,0.031000,0.031000
CLÉZIO GAUGLITZ,0.031000,0.031000
FABIANA ANTONICCI,0.030500,0.030500
MARISA MICHELACCI VALLEJO,0.030200,0.030200


,txtObjetoContrato
360,NaN


## 1.5 Estágios

In [105]:
estagios_filter = pessoal_filter & (pessoal['txtDescricaoItemDespesa'].str.contains('Estagiários', case=False))
estagios = (pessoal[estagios_filter])
empenhos_df.loc[estagios_filter, 'cat_2'] = 'estagios'
display(len(empenhos_df[empenhos_df['cat_2'] == 'estagios']) == len(estagios))
groupSum(estagios)

/home/bpl/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


True

,valTotalEmpenhado,valLiquidado
txtCategoriaEconomica,,
Despesas Correntes,42.625777,33.448145


,valTotalEmpenhado,valLiquidado
txtDescricaoElemento,,
Outros Serviços de Terceiros - Pessoa Jurídica,34.734801,27.796770
Outros Auxílios Financeiros a Pessoas Físicas,7.890976,5.651375


,valTotalEmpenhado,valLiquidado
txtDescricaoFonteRecurso,,
Tesouro Municipal,42.625777,33.448145


...

,valTotalEmpenhado,valLiquidado
txtDescricaoUnidade,,
Gabinete do Secretário,42.625777,33.448145


,valTotalEmpenhado,valLiquidado
txtGrupoDespesa,,
Outras Despesas Correntes,42.625777,33.448145


,valTotalEmpenhado,valLiquidado
txtModalidadeAplicacao,,
Transferências a Instituições Privadas sem Fins Lucrativos,42.625777,33.448145


,valTotalEmpenhado,valLiquidado
txtRazaoSocial,,
CENTRO DE INTEGRAÇÃO EMPRESA ESCOLA CIEE,42.625777,33.448145


,txtObjetoContrato
8005,NaN


# 2. Transporte Escolar

## 2.1. Passe livre

#### Como filtrar? Acredito que este gasto deva estar fora do orçamento da SME.

In [106]:
# encontrar repasses para SMT?
(empenhos_df[empenhos_df['txtRazaoSocial'].str.contains('transporte', case=False)]
 .groupby('txtRazaoSocial')['valLiquidado']
 .sum().apply(lambda x: round(x / 1000000, 1))
 .sort_values(ascending=False)
 .head(20)
)

txtRazaoSocial
TB SERVIÇOS TRANSPORTE LIMPEZA GERENCIAMENTO E RECURSOS HUMANOS S/A     21.9
SERVIDORES DA SECRETARIA MUNICIPAL DE EDUCAÇÃO - AUX. TRANSPORTE        17.8
COOPERATIVA BRASILEIRA DE TRANSPORTE COBRATE                             6.3
INTEGRA SOLUCOES EM LOGISTICA E TRANSPORTE LTDA - EPP                    4.7
P R SELIS TRANSPORTES EIRELI - EPP                                       2.4
COOPERATIVA DE TRANSPORTE ESCOLAR DE CONDUTOR E MONITOR - COOTRECM       1.9
AÇÃO TRANSPORTES E TURISMO LTDA - EPP                                    1.7
BELLA TRANSPORTES LTDA - ME                                              1.3
JULIEDRY TRANSPORTES LTDA -ME                                            1.3
VIA LUZ TRANSPORTES DE PASSAGEIROS LTDA                                  1.3
JTI TRANSPORTES LTDA - ME                                                1.2
TRANSRUBY TRANSPORTES EM GERAL LTDA - ME                                 1.0
WM DIAS TRANSPORTE ESCOLAR LTDA ME                           

## 2.2. TEG

#### Checar filtro

In [107]:
teg_filter = (
    (empenhos_df['txtDescricaoProjetoAtividade'].str.contains('transporte', case=False))
)
teg = (empenhos_df[teg_filter])
empenhos_df.loc[teg_filter, 'cat_1'] = 'transporte'
empenhos_df.loc[teg_filter, 'cat_2'] = 'teg'
groupSum(teg)

,valTotalEmpenhado,valLiquidado
txtCategoriaEconomica,,
Despesas Correntes,233.014406,194.595252


,valTotalEmpenhado,valLiquidado
txtDescricaoElemento,,
Outros Serviços de Terceiros - Pessoa Física,123.252825,103.753417
Outros Serviços de Terceiros -Pessoa Jurídica,103.894366,86.712105
Obrigações Tributárias e Contributivas,5.515257,3.788212
Despesas de Exercícios Anteriores,0.351958,0.341519


,valTotalEmpenhado,valLiquidado
txtDescricaoFonteRecurso,,
Tesouro Municipal,233.014406,194.595252


,valTotalEmpenhado,valLiquidado
txtDescricaoItemDespesa,,
Transporte Escolar - EMEF,226.955018,190.285810
INSS / Contribuinte Individual,5.515257,3.788212
Outros Serviços de Terceiros - Pessoa Jurídica - Tributável,0.276233,0.273998
Pequenas Reformas - Centros Esportivos e Similares,0.112273,0.103859
Outros Serviços de Terceiros - Pessoa Física,0.075726,0.067521
Transporte Escolar - EMEI,0.065014,0.065014
Serviço com a População de Rua,0.007867,0.007867
Transporte Escolar - EMEE,0.002972,0.002972
Veículos,0.004048,0.000000


,valTotalEmpenhado,valLiquidado
txtDescricaoPrograma,,
Melhoria da qualidade e ampliação do acesso à educação,233.014406,194.595252


...

,valTotalEmpenhado,valLiquidado
txtDescricaoUnidade,,
Gabinete do Secretário,233.014406,194.595252


,valTotalEmpenhado,valLiquidado
txtGrupoDespesa,,
Outras Despesas Correntes,233.014406,194.595252


,valTotalEmpenhado,valLiquidado
txtModalidadeAplicacao,,
Aplicações Diretas,233.014406,194.595252


,valTotalEmpenhado,valLiquidado
txtRazaoSocial,,
COOPERATIVA BRASILEIRA DE TRANSPORTE COBRATE,7.420587,6.290727
"COOPER- X - COOP SERV TRANSPORTAD AUT PAS, ESCOLARES E MON DO EST SP",6.522607,5.387077
INSTITUTO NACIONAL DO SEGURO SOCIAL,5.515257,3.788212
P R SELIS TRANSPORTES EIRELI - EPP,2.994445,2.366641
COOPER DE TRAB. DOS C. A.T.P NA REP. FED.DO.BR COOPERUNIÃO BRASILEIRA,2.403070,2.084142
COOPERATIVA DE TRANSPORTE ESCOLAR DE CONDUTOR E MONITOR - COOTRECM,2.205080,1.903273
BELLA TRANSPORTES LTDA - ME,1.472303,1.322393
JULIEDRY TRANSPORTES LTDA -ME,1.518641,1.299755
VIA LUZ TRANSPORTES DE PASSAGEIROS LTDA,1.381945,1.258995


,txtObjetoContrato
2149,NaN
2218,REAPASSE DE RECURSSOS A APM - PTRF - CUSTEIO E CAPITAL/ 2017
2234,CONTRATAÇÃO DE AGENTE DE RECREAÇÃO RECREIO NAS FERIAS /2017
2374,PAGAMENTO DO ADITAMENTO DO CEI LAR CRIANÇA FELIZ
2476,"Repasse de recursos para APM de acordo com a Lei Federal nº 9.394/1996, Lei Municipal nº 13.991/..."
2510,CONTRATAÇÃO DE AGENTE DE RECREAÇÃO - RECREIO NAS FÉRIAS JULHO / 2017
2519,"Repasse de recursos para APM de acordo com a Lei Federal nº 9.394/1996, Lei Municipal nº 13.991/..."
2527,IT - 1º REPASSE PARA APM DA EMEF DR. JOSE PEDRO LEITE CORDEIRO - DE ACORDO COM A LEI Nº 9394//96...
2542,SERVIÇO DE PODA (S) E REMOÇÃO NA EMEF JEAN MERMOZ.
2555,papel sulfite A4


## 2.3 Aluguel de veículos

In [108]:
veiculos_filter = (
    (empenhos_df['txtDescricaoSubElemento'].str
     .contains('Locação de Bens Móveis de Outra Natureza e Intangíveis', case=False)
     &
     ~(empenhos_df['txtDescricaoItemDespesa'].str
     .contains('Locação de Bens Móveis com Finalidade Artística e Cultural', case=False)
    ))
)
veiculos = (empenhos_df[veiculos_filter])
empenhos_df.loc[veiculos_filter, 'cat_1'] = 'transporte'
empenhos_df.loc[veiculos_filter, 'cat_2'] = 'veiculos'
groupSum(veiculos)

,valTotalEmpenhado,valLiquidado
txtCategoriaEconomica,,
Despesas Correntes,23.468506,15.920269


,valTotalEmpenhado,valLiquidado
txtDescricaoElemento,,
Outros Serviços de Terceiros -Pessoa Jurídica,23.468506,15.920269


,valTotalEmpenhado,valLiquidado
txtDescricaoFonteRecurso,,
Tesouro Municipal,23.468506,15.920269


,valTotalEmpenhado,valLiquidado
txtDescricaoItemDespesa,,
Veículos Leves com Motorista,13.177409,9.390203
Ônibus com Motorista,5.484531,3.555379
Veículos Pesados com Motorista,3.201524,1.763547
Veículos Pesados,1.155082,0.982007
Veículos Leves,0.427569,0.229134
Ônibus,0.022391,0.000000


,valTotalEmpenhado,valLiquidado
txtDescricaoPrograma,,
Suporte Administrativo,18.068870,12.429799
Melhoria da qualidade e ampliação do acesso à educação,5.399636,3.490471


...

,valTotalEmpenhado,valLiquidado
txtDescricaoUnidade,,
Diretoria Regional de Educação - Campo Limpo,4.371954,3.036236
Gabinete do Secretário,2.531521,1.686781
Diretoria Regional de Educação - Penha,1.698923,1.258398
Departamento de Alimentação Escolar,1.144227,1.083600
Diretoria Regional de Educação - São Miguel,1.223700,1.062067
Diretoria Regional de Educação - Santo Amaro,1.726081,1.036800
Diretoria Regional de Educação - Itaquera,1.327189,0.933929
Diretoria Regional de Educação - Ipiranga,1.228475,0.912538
Diretoria Regional de Educação - Butantã,1.127668,0.829937


,valTotalEmpenhado,valLiquidado
txtGrupoDespesa,,
Outras Despesas Correntes,23.468506,15.920269


,valTotalEmpenhado,valLiquidado
txtModalidadeAplicacao,,
Aplicações Diretas,23.468506,15.920269


,valTotalEmpenhado,valLiquidado
txtRazaoSocial,,
EBSL EMPRESA BRASILEIRA DE SERVICOS DE LOCACOES LTDA - EPP,2.774193,1.933145
AÇÃO TRANSPORTES E TURISMO LTDA - EPP,3.302512,1.725778
UNITRANSP - COOP. UNIÃO INTERMODAL DE TRANSP. AUTÔNOMOS DE SÃO PAULO,1.882072,1.482244
TRANSRUBY TRANSPORTES EM GERAL LTDA - ME,1.329792,1.033230
TZAR LOGÍSTICA LTDA,1.100882,0.982007
DANLEX SERVIÇOS LTDA.,2.115157,0.948752
COOPER PLANALTO - COOPERATIVA DE TRABALHO EM TRANSPORTES RODOVIÁRIOS,1.157570,0.898686
CTPT - COOP. DE TRAB. DOS PROF. DA ÁREA DE TRANSPORTES,1.310132,0.895729
TOP 5 SOLUCOES EM TRANSPORTES LTDA.- ME,0.810024,0.679812


,txtObjetoContrato
644,SP/MP CONTRATAÇÃO DE EMPRESA ESPECIALIZADA NA INTERMEDIAÇÃO DE SERVIÇOS DE TRANSPORTE INDIVIDUAL...
645,NaN
647,REPASSES DE RECURSOS PARA APM DE ACORDO COM: LEI FEDERAL Nº 9394/1996; LEI MUNICIPAL Nº 13.991/0...
658,SP/MP CONTRATAÇÃO DE EMPRESA ESPECIALIZADA NA LOCAÇÃO DE CAMINHÕES TOCO E VUC PARA A DIRETORIA R...
680,contratação através de ATA DE RP Nº 02/SME/2017 – CONTRATAÇÃO DE LOCAÇÃO DE ÔNIBUS COM MOTORISTA...
1130,"Empresa especializada na intermediação ou agenciamento de serviços de transporte individual, via..."
4928,Contratação de serviços de transporte em ônibus de fretamento de viagem - ATA de RP nº 06/SME/2017
6203,Contratação de transporte de alunos da Rede Municipal de Educação para o evento JAM de Robótica ...
6337,CONTRATAÇÃO DE ÔNIBUS COM MOTORISTA E COMBUSTÍVEL PARA ATENDIMENTO DA DEMANDA DA DRE IPIRANGA.
6338,Contratação de Ônibus para o 1º Mutirão do Programa Visão do Futuro


## 2.3. Outros?

# 3. Funcionamento das escolas

## 3.1. Limpeza

#### Como filtrar?

In [109]:
limpeza_filter = (
    (empenhos_df['txtDescricaoItemDespesa'].str.contains('limpeza', case=False))
    |
    (empenhos_df['txtDescricaoSubElemento'].str.contains('limpeza', case=False))
)
limpeza = (empenhos_df[limpeza_filter])
empenhos_df.loc[limpeza_filter, 'cat_1'] = 'funcionamento_unidades'
empenhos_df.loc[limpeza_filter, 'cat_2'] = 'limpeza'
groupSum(limpeza)

,valTotalEmpenhado,valLiquidado
txtCategoriaEconomica,,
Despesas Correntes,465.139461,407.756859


,valTotalEmpenhado,valLiquidado
txtDescricaoElemento,,
Outros Serviços de Terceiros -Pessoa Jurídica,460.595615,404.679844
Material de Consumo,3.827840,2.588546
Locação de Mão-de-Obra,0.716006,0.488469


,valTotalEmpenhado,valLiquidado
txtDescricaoFonteRecurso,,
Tesouro Municipal,465.139461,407.756859


,valTotalEmpenhado,valLiquidado
txtDescricaoItemDespesa,,
Limpeza de Ambientes,460.588315,404.674394
Produtos para Toucador e Higiene Pessoal,2.349727,1.613706
Diversos Materiais de Limpeza e Higienização,1.362784,0.871918
Limpeza e Conservação,0.716006,0.488469
Antisépticos e Desinfetantes,0.115328,0.102923
"Dedetização, Descupinização e Outros",0.007300,0.005450


,valTotalEmpenhado,valLiquidado
txtDescricaoPrograma,,
Melhoria da qualidade e ampliação do acesso à educação,460.024135,403.621124
Suporte Administrativo,5.004523,4.034690
"Participação, transparência e controle social da administração pública",0.110803,0.101046


...

,valTotalEmpenhado,valLiquidado
txtDescricaoUnidade,,
Gabinete do Secretário,458.435946,402.873881
Diretoria Regional de Educação - São Miguel,1.320629,0.729638
Diretoria Regional de Educação Capela do Socorro,0.770271,0.686966
Diretoria Regional de Educação - Campo Limpo,0.950557,0.584489
Diretoria Regional de Educação - Penha,0.645757,0.535053
Diretoria Regional de Educação - Guaianases,0.698750,0.522635
Diretoria Regional de Educação - São Mateus,0.459895,0.406673
Diretoria Regional de Educação - Butantã,0.322566,0.302487
Diretoria Regional de Educação - Jaçanã/Tremembé,0.328513,0.280679


,valTotalEmpenhado,valLiquidado
txtGrupoDespesa,,
Outras Despesas Correntes,465.139461,407.756859


,valTotalEmpenhado,valLiquidado
txtModalidadeAplicacao,,
Aplicações Diretas,465.139461,407.756859


,valTotalEmpenhado,valLiquidado
txtRazaoSocial,,
LIMPADORA CALIFORNIA LTDA,56.133925,44.034916
ARCOLIMP SERVIÇOS GERAIS LTDA,42.049079,39.114548
G4S INTERATIVA SERVICE LTDA,35.803843,33.251570
"GUIMA CONSECO CONSTRUÇÃO, SERVIÇOS E COMÉRCIO LTDA",31.047000,28.949667
DEMAX SERVICOS E COMERCIO LTDA,30.061606,28.161345
A. TONANNI CONSTRUÇÕES E SERVIÇOS LTDA,25.723431,23.657501
EPS - EMPRESA PAULISTA DE SERVICOS S.A.,24.503134,22.728663
GOCIL SERVICOS GERAIS LTDA,32.003199,22.321998
TB SERVIÇOS TRANSPORTE LIMPEZA GERENCIAMENTO E RECURSOS HUMANOS S/A,22.812355,21.183561


,txtObjetoContrato
340,SP/SR - AQUISIÇÃO DE ÁLCOOL GEL PARA ATENDIMENTO DAS NECESSIDADES DAS UNIDADES ESCOLARES JURISD...
568,SP/MP AQUISIÇÃO DE MATERIAIS PARA LIMPEZA PARA O CEFOR
612,"REPASSES DO RECURSO PARA A APM DE ACORDO COM A LEI FEDERAL 9394/96, LEI MUNICIPAL 13991/05, DE..."
614,NaN
617,"Prestação de serviços de limpeza, asseio e conservação predial, visando a obtenção de adequadas ..."
1118,Emergencial Limpeza Administrativa
1621,AQUISIÇÃO DE ÁGUA SANITÁRIA
1622,AQUISIÇÃO DE ALCOOL GEL
1624,AQUISIÇÃO DE CREME OU GEL DENTAL INFANTIL
1632,AQUISIÇÃO DE LUVA DESCARTÁVEL


## 3.2. Vigilância

#### Como filtrar?

In [110]:
vigilancia_filter = (
    (empenhos_df['txtDescricaoItemDespesa'].str.contains('vigilância', case=False))
    |
    (empenhos_df['txtDescricaoItemDespesa'].str.contains('vigilancia', case=False))
)
vigilancia = (empenhos_df[vigilancia_filter])
empenhos_df.loc[vigilancia_filter, 'cat_1'] = 'funcionamento_unidades'
empenhos_df.loc[vigilancia_filter, 'cat_2'] = 'vigilancia'
groupSum(vigilancia)

,valTotalEmpenhado,valLiquidado
txtCategoriaEconomica,,
Despesas Correntes,215.862453,181.857045


,valTotalEmpenhado,valLiquidado
txtDescricaoElemento,,
Outros Serviços de Terceiros -Pessoa Jurídica,214.285475,180.449088
Locação de Mão-de-Obra,1.576978,1.407958


,valTotalEmpenhado,valLiquidado
txtDescricaoFonteRecurso,,
Tesouro Municipal,215.862453,181.857045


,valTotalEmpenhado,valLiquidado
txtDescricaoItemDespesa,,
"Guarda, Vigilância e Segurança em Geral",214.285475,180.449088
Vigilância Ostensiva,1.576978,1.407958


,valTotalEmpenhado,valLiquidado
txtDescricaoPrograma,,
Melhoria da qualidade e ampliação do acesso à educação,203.620787,171.500223
Suporte Administrativo,12.013376,10.148020
"Participação, transparência e controle social da administração pública",0.228290,0.208802


...

,valTotalEmpenhado,valLiquidado
txtDescricaoUnidade,,
Gabinete do Secretário,205.742407,174.029004
Diretoria Regional de Educação - Freguesia/Brasilândia,2.017294,1.182547
Diretoria Regional de Educação - Santo Amaro,1.377752,1.125484
Diretoria Regional de Educação - Campo Limpo,1.160609,0.992040
Diretoria Regional de Educação - Jaçanã/Tremembé,1.116469,0.973041
Diretoria Regional de Educação - São Miguel,0.973168,0.770355
Diretoria Regional de Educação - Ipiranga,0.837397,0.767614
Diretoria Regional de Educação - Pirituba,0.840800,0.572334
Diretoria Regional de Educação - São Mateus,0.645336,0.549133


,valTotalEmpenhado,valLiquidado
txtGrupoDespesa,,
Outras Despesas Correntes,215.862453,181.857045


,valTotalEmpenhado,valLiquidado
txtModalidadeAplicacao,,
Aplicações Diretas,215.862453,181.857045


,valTotalEmpenhado,valLiquidado
txtRazaoSocial,,
ALBATROZ SEGURANÇA E VIGILÂNCIA LTDA,27.713410,24.890311
G4S VANGUARDA SEGURANCA E VIGILANCIA LTDA,27.801421,24.259330
PLURI SEGURANÇA E VIGILANCIA LTDA,24.536613,22.139465
SEAL SEGURANCA ALTERNATIVA EIRELI,25.595870,18.863292
LÓGICA SEGURANÇA E VIGILÂNCIA EIRELI,20.400099,18.448181
POWER - SEGURANÇA E VIGILANCIA LTDA,22.136168,14.820399
GOCIL SERVIÇOS DE VIGILÂNCIA E SEGURANÇA LTDA,15.087489,13.067197
SUPORTE SERVIÇOS DE SEGURANÇA LTDA,10.040263,9.180736
ALSA FORT SEGURANCA LTDA,7.786121,6.402367


,txtObjetoContrato
601,SP/MP CONTRATAÇÃO DE SERVIÇOS DE SEGURANÇA E VIGILÂNCIA DESARMADA PARA A DRE-MP SENDO 5 POSTOS D...
602,NaN
7283,LOCAÇÃO DE EMPRESA PARA PRESTAÇÃO DE SERVIÇO DE MINUTAGEM - PABX - DRE
7284,"Serviços de vigilância CEUs - lotes 01, 02, 04 e 05"
7326,"Serviços de vigilância CEUs - lotes 03, 06 e 07"
24297,AQUISIÇÃO DE COLCHONETE - RECREIO NAS FÉRIAS - JULHO/2017
35298,PROGRAMA RECREIO NAS FÉRIAS - JANEIRO/2017 - DE ACORDO COM O EDITAL DE CREDENCIAMENTO Nº 004/201...
37436,Contratação de Empresa para Serviços de Manutenção na EMEF Dom Pedro I
37438,PREGÃO ELETRÔNICO 12/2017 - CONTRATAÇÃO DE EMPRESA PARA PRESTAÇ~]AO DE SERVIÇOS DE SEGURANÇA E ...
39742,Contratação de empresa para serviços de segurança para eventos


## 3.4 Água, esgoto, energia, telefonia

In [111]:
agua_filter = (
    (empenhos_df['txtDescricaoItemDespesa'].str.contains('água e esgoto', case=False))
    |
    (empenhos_df['txtDescricaoItemDespesa'].str.contains('água e sistema de esgoto', case=False))
)
agua = (empenhos_df[agua_filter])
empenhos_df.loc[agua_filter, 'cat_1'] = 'funcionamento_unidades'
empenhos_df.loc[agua_filter, 'cat_2'] = 'agua_esgoto'
groupSum(agua)

,valTotalEmpenhado,valLiquidado
txtCategoriaEconomica,,
Despesas Correntes,126.319533,93.020801


,valTotalEmpenhado,valLiquidado
txtDescricaoElemento,,
Outros Serviços de Terceiros -Pessoa Jurídica,126.318075,93.019344
Despesas de Exercícios Anteriores,0.001458,0.001458


,valTotalEmpenhado,valLiquidado
txtDescricaoFonteRecurso,,
Tesouro Municipal,126.319533,93.020801


,valTotalEmpenhado,valLiquidado
txtDescricaoItemDespesa,,
Água e Esgoto,126.312290,93.019344
Serviços de Abastecimento de Água e Sistema de Esgoto Sanitário,0.001458,0.001458
Reembolso de Água e Esgoto,0.005785,0.000000


,valTotalEmpenhado,valLiquidado
txtDescricaoPrograma,,
Melhoria da qualidade e ampliação do acesso à educação,123.812070,92.135426
Suporte Administrativo,2.507463,0.885375


...

,valTotalEmpenhado,valLiquidado
txtDescricaoUnidade,,
Diretoria Regional de Educação - Campo Limpo,19.281355,15.431377
Diretoria Regional de Educação - Pirituba,11.278434,8.917984
Diretoria Regional de Educação - São Mateus,11.432222,8.053293
Diretoria Regional de Educação - São Miguel,10.062024,7.446496
Diretoria Regional de Educação - Penha,7.175886,6.907916
Diretoria Regional de Educação - Ipiranga,10.065321,6.769633
Diretoria Regional de Educação - Santo Amaro,8.827468,6.516152
Diretoria Regional de Educação - Guaianases,7.473731,6.136073
Diretoria Regional de Educação - Itaquera,7.324075,5.698005


,valTotalEmpenhado,valLiquidado
txtGrupoDespesa,,
Outras Despesas Correntes,126.319533,93.020801


,valTotalEmpenhado,valLiquidado
txtModalidadeAplicacao,,
Aplicações Diretas,126.319533,93.020801


,valTotalEmpenhado,valLiquidado
txtRazaoSocial,,
CIA DE SANEAMENTO BÁSICO DO ESTADO DE SÃO PAULO SABESP,123.805741,93.020801
ELETROPAULO METROPOLITANA ELETRICIDADE DE SÃO PAULO S.A.,2.496292,0.000000
TELEFÔNICA BRASIL S.A.,0.017500,0.000000


,txtObjetoContrato
347,SP/SR CONTRATAÇÃO DE SERVIÇOS DE AGUA E ESGOSTO REFERENTE O EXERCÍCIO DE 2017 PARA A UNIDADES DR...
349,"SP/SR FORNECIMENTO DE SERVIÇOS DE AGUA E ESGOSTO PARA O CEU PQ SÃO CARLOS, CEU PQ VEREDAS, CEU T..."
351,SP/SR FORNECIMENTO DE SERVIÇOS DE AGUA E ESGOSTO PARA O CEU'S DA DRE/MP
1690,PAGTO. DE CONCESSIONÁRIA - SABESP
6166,Pagamento dos serviços de fornecimento de água e esgoto para as Unidades Centrais da SME
14291,Consumode água e esgoto da DRE Pirituba e das Unidades Educacionais a ela subordinadas.
14301,Consumo de energia elétrica da DRE Pirituba e das Unidades Escolares a ela subordinadas.
19364,"CONSUMO DE ÁGUA - SABESP - CEI, EMEI, EMEF, CIEJA, ADM E CEU."
19371,SERVIÇOS DE ENERGIA ELETRICA - ELETROPAULO
20169,NaN


In [112]:
eletricidade_filter = (empenhos_df['txtRazaoSocial'].str.contains('eletropaulo', case=False))
eletricidade = (empenhos_df[eletricidade_filter])
empenhos_df.loc[eletricidade_filter, 'cat_1'] = 'funcionamento_unidades'
empenhos_df.loc[eletricidade_filter, 'cat_2'] = 'eletricidade'
groupSum(eletricidade)

,valTotalEmpenhado,valLiquidado
txtCategoriaEconomica,,
Despesas Correntes,66.003813,44.973127


,valTotalEmpenhado,valLiquidado
txtDescricaoElemento,,
Outros Serviços de Terceiros -Pessoa Jurídica,66.003813,44.973127


,valTotalEmpenhado,valLiquidado
txtDescricaoFonteRecurso,,
Tesouro Municipal,66.003813,44.973127


,valTotalEmpenhado,valLiquidado
txtDescricaoItemDespesa,,
Energia Elétrica de Prédios Públicos,63.432221,44.935585
Serviços de Instalação de Rede de Energia,0.040100,0.037542
"Desmontagem, Remoção e Montagem de Equipamentos e Afins",0.001164,0.000000
Gás Canalizado,0.034000,0.000000
Reembolso de Energia Elétrica,0.000035,0.000000
Água e Esgoto,2.496292,0.000000


,valTotalEmpenhado,valLiquidado
txtDescricaoPrograma,,
Melhoria da qualidade e ampliação do acesso à educação,63.071609,43.520762
Suporte Administrativo,2.932204,1.452365


...

,valTotalEmpenhado,valLiquidado
txtDescricaoUnidade,,
Diretoria Regional de Educação - Campo Limpo,7.175809,6.192868
Diretoria Regional de Educação - São Miguel,7.582278,4.555495
Diretoria Regional de Educação - Pirituba,5.568449,4.396149
Diretoria Regional de Educação - São Mateus,7.733360,4.101346
Diretoria Regional de Educação - Guaianases,4.292843,3.463225
Diretoria Regional de Educação - Itaquera,4.202624,3.288448
Diretoria Regional de Educação - Penha,4.352793,3.090721
Diretoria Regional de Educação - Ipiranga,3.446775,2.829503
Diretoria Regional de Educação - Freguesia/Brasilândia,3.675465,2.675959


,valTotalEmpenhado,valLiquidado
txtGrupoDespesa,,
Outras Despesas Correntes,66.003813,44.973127


,valTotalEmpenhado,valLiquidado
txtModalidadeAplicacao,,
Aplicações Diretas,66.003813,44.973127


,valTotalEmpenhado,valLiquidado
txtRazaoSocial,,
ELETROPAULO METROPOLITANA ELETRICIDADE DE SÃO PAULO S.A.,66.003813,44.973127


,txtObjetoContrato
343,SP/SR SERVIÇOS DE FORNECIMENTO DE ENERGIA ELETRICA REFERENTE AO EXERCÍCIO DE 2017 PARA A UNIDADE...
346,SP/SR SERVIÇOS DE FORNECIMENTO DE ENERGIA ELETRICA REFERENTE AO EXERCÍCIO DE 2017 PARA OS CEU'S ...
1680,PAGTO. DE CONCESSIONÁRIA - ELETROPAULO
6165,Pagamento dos serviços de fornecimento de energia elétrica para as Unidades Centrais da SME
13939,NaN
14273,Consumo de energia elétrica da DRE Pirituba e das Unidades Escolares a ela subordinadas.
19339,SERVIÇOS DE ENERGIA ELETRICA - ELETROPAULO
22725,CONCESSIONÁRIA AES ELETROPAULO 2017
24175,CONTRATAÇÃO DE EMPRESA PARA FORNECIMENTO DE ENERGIA ELÉTRICA - 2017
28392,CONSUMO E TARIFAS DE ENERGIA ELÉTRICA - CEUS


In [113]:
telefone_filter = (empenhos_df['txtDescricaoItemDespesa'].str.contains('telefonia', case=False))
telefone = (empenhos_df[telefone_filter])
empenhos_df.loc[telefone_filter, 'cat_1'] = 'funcionamento_unidades'
empenhos_df.loc[telefone_filter, 'cat_2'] = 'telefone'
groupSum(telefone)

,valTotalEmpenhado,valLiquidado
txtCategoriaEconomica,,
Despesas Correntes,25.054313,16.570464


,valTotalEmpenhado,valLiquidado
txtDescricaoElemento,,
Outros Serviços de Terceiros -Pessoa Jurídica,25.054313,16.570464


,valTotalEmpenhado,valLiquidado
txtDescricaoFonteRecurso,,
Tesouro Municipal,25.054313,16.570464


,valTotalEmpenhado,valLiquidado
txtDescricaoItemDespesa,,
"Serviço de Instalação, Ampliação de Sist. de Telefonia/Teleinformática",15.433389,11.321042
Telefonia Fixa,9.162018,5.216719
Telefonia Móvel,0.458906,0.032704


,valTotalEmpenhado,valLiquidado
txtDescricaoPrograma,,
Modernização e descentralização da gestão e prestação de serviços,15.368131,11.271799
Melhoria da qualidade e ampliação do acesso à educação,8.283374,4.892816
Suporte Administrativo,1.401059,0.405849
"Participação, transparência e controle social da administração pública",0.001749,0.000000


...

,valTotalEmpenhado,valLiquidado
txtDescricaoUnidade,,
Gabinete do Secretário,15.705322,11.287676
Diretoria Regional de Educação - Campo Limpo,1.098561,0.685051
Diretoria Regional de Educação - Jaçanã/Tremembé,0.727494,0.516987
Diretoria Regional de Educação - Guaianases,0.814640,0.510620
Diretoria Regional de Educação - São Miguel,0.769980,0.493618
Diretoria Regional de Educação - Pirituba,0.616080,0.466939
Diretoria Regional de Educação - Ipiranga,0.664515,0.455821
Diretoria Regional de Educação - São Mateus,0.579139,0.449281
Diretoria Regional de Educação - Penha,0.596295,0.425594


,valTotalEmpenhado,valLiquidado
txtGrupoDespesa,,
Outras Despesas Correntes,25.054313,16.570464


,valTotalEmpenhado,valLiquidado
txtModalidadeAplicacao,,
Aplicações Diretas,25.054313,16.570464


,valTotalEmpenhado,valLiquidado
txtRazaoSocial,,
PRIMESYS SOLUÇÕES EMPRESARIAIS S.A.,14.698120,10.708426
TELEFÔNICA BRASIL S.A.,9.974875,5.550734
NEC LATIN AMERICA S.A.,0.377138,0.310331
CLARO S.A.,0.004180,0.000973


,txtObjetoContrato
352,NaN
354,SP/SR CONTRATAÇÃO DE SERVIÇOS DE TELEFONIA FIXA REFERENTE AO EXERCÍCIO DE 2017 PARA AS UNIDADES ...
359,SP/SR CONTRATAÇÃO DE SERVIÇOS DE TELEFONIA FIXA REFERENTE AO EXERCÍCIO DE 2017 PARAAS UNIDADES D...
638,MANUTENÇÃO E CONSERVAÇÃO - CEU PARAISÓPOLIS
1109,AQUISIÇÃO DE ÓLEO DE SOJA
1699,PAGTO. DE CONCESSIONÁRIA - TELEFÔNICA
12235,Aquisição de Luva de vivil
12240,CONSUMO DE TELEFONIA FIXA - DRE
14308,Consumo de telefonia fixa das Unidades Escolares da DRE Pirituba/Jaraguá.
19393,SERVIÇOS DE TELEFONIA-TELEFONICA


## 3.5 Mobiliário

In [114]:
mobiliario_filter = (empenhos_df['txtDescricaoSubElemento'].str.contains('Mobiliário', case=False))
mobiliario = (empenhos_df[mobiliario_filter])
empenhos_df.loc[mobiliario_filter, 'cat_1'] = 'funcionamento_unidades'
empenhos_df.loc[mobiliario_filter, 'cat_2'] = 'mobiliario'
groupSum(mobiliario)

,valTotalEmpenhado,valLiquidado
txtCategoriaEconomica,,
Despesas de Capital,10.793645,0.011702


,valTotalEmpenhado,valLiquidado
txtDescricaoElemento,,
Equipamentos e Material Permanente,10.793645,0.011702


,valTotalEmpenhado,valLiquidado
txtDescricaoFonteRecurso,,
Tesouro Municipal,10.793645,0.011702


,valTotalEmpenhado,valLiquidado
txtDescricaoItemDespesa,,
Outros Móveis em Geral,0.007947,0.007947
Não Padronizados,0.003755,0.003755
Carteiras e Cadeiras Escolares,5.986720,0.000000
Padronizados,4.795223,0.000000


,valTotalEmpenhado,valLiquidado
txtDescricaoPrograma,,
Melhoria da qualidade e ampliação do acesso à educação,10.793645,0.011702


...

,valTotalEmpenhado,valLiquidado
txtDescricaoUnidade,,
Diretoria Regional de Educação - Guaianases,0.007947,0.007947
Diretoria Regional de Educação - Butantã,0.003755,0.003755
Gabinete do Secretário,10.781943,0.000000


,valTotalEmpenhado,valLiquidado
txtGrupoDespesa,,
Investimentos,10.793645,0.011702


,valTotalEmpenhado,valLiquidado
txtModalidadeAplicacao,,
Aplicações Diretas,10.793645,0.011702


,valTotalEmpenhado,valLiquidado
txtRazaoSocial,,
SPEZIA COMERCIAL DE MATERIAIS E SUPRIMENTOS EIRELI - ME,0.004447,0.004447
GHAIA COMERCIAL LTDA-EPP,0.003755,0.003755
INNOVARY FUTURE COMERCIAL LTDA - ME,0.003500,0.003500
ANDRESSA PANINI ALBISSU EPP,0.362138,0.000000
ARTEMOVEIS - SOLUCOES E COMERCIO DE MOVEIS LTDA - ME,0.080364,0.000000
C.G. RANDI MOVEIS PARA ESCRITORIO - EPP,0.062990,0.000000
CANTARES MAGAZINE - EIRELI,0.329306,0.000000
EQUISCOLA EQUIPAMENTOS ESCOLARES LTDA-ME,0.418824,0.000000
ERGO MOBILI IND. E COM. DE MOVEIS LTDA,2.245002,0.000000


,txtObjetoContrato
5317,AQUISIÇÃO DE MOBILIARIO ESCOLAR
5318,"Aquisição de Mobiliário ""Sala de Aula EMEF - Tamanho I e Refeitório Escolar - Tamanho I"""
5319,Aquisição de Mobiliário - Conjunto para Professor
5322,Aquisição Mobiliário Administrativo - Mesa comum para Secretaria
5325,Aquisição de mobiliario
5328,Aquisição de Mobiliário Escolar
5329,Aquisição de 540 Armários de Madeira de 02 (dois) Corpos
5337,Aquisição de conjunto sala de aula EMEI
5341,Aquisição de Conjunto Sala de Leitura EMEF Tam. I e II
5342,Aquisição de cadeiras diversas


## 3. 5 Manutenção

In [115]:
manutencao_filter = (
    (empenhos_df['txtDescricaoSubElemento'].str.contains('Manut', case=False))
    &
    ~(empenhos_df['txtDescricaoSubElemento'].str.contains('processamento de dados', case=False))
    &
    ~(empenhos_df['txtDescricaoSubElemento'].str.contains('software', case=False))
)
manutencao = (empenhos_df[manutencao_filter])
empenhos_df.loc[manutencao_filter, 'cat_1'] = 'funcionamento_unidades'
empenhos_df.loc[manutencao_filter, 'cat_2'] = 'manutencao'
groupSum(manutencao)

,valTotalEmpenhado,valLiquidado
txtCategoriaEconomica,,
Despesas Correntes,25.59243,18.071561
Despesas de Capital,1.14000,0.569343


,valTotalEmpenhado,valLiquidado
txtDescricaoElemento,,
Outros Serviços de Terceiros -Pessoa Jurídica,25.493945,18.017075
Outros Serviços de Terceiros - Pessoa Jurídica,1.140000,0.569343
Material de Consumo,0.098486,0.054486


,valTotalEmpenhado,valLiquidado
txtDescricaoFonteRecurso,,
Tesouro Municipal,26.73243,18.640903


,valTotalEmpenhado,valLiquidado
txtDescricaoItemDespesa,,
Elevadores,9.009160,6.225331
Outras Máquinas e Equipamentos,4.090783,3.732811
Pequenas Reformas - Escolas de Ensino Fundamental,3.019048,2.532999
Pequenas Reformas - Escolas de Educação Infantil,2.853613,2.227234
Pequenas Reformas - Centros de Educação Infantil,1.656765,1.274057
Pequenas Reformas - Outros Imóveis,0.888097,0.687964
Manutenção e Reformas de Bens Imóveis,1.140000,0.569343
Equipamentos de Processamento de Dados,0.583397,0.522691
Manutenção e Conservação de Bens Imóveis,0.529771,0.173759


,valTotalEmpenhado,valLiquidado
txtDescricaoPrograma,,
Melhoria da qualidade e ampliação do acesso à educação,23.749487,16.438462
Suporte Administrativo,1.846567,1.218933
Modernização e descentralização da gestão e prestação de serviços,1.136377,0.983508


...

,valTotalEmpenhado,valLiquidado
txtDescricaoUnidade,,
Gabinete do Secretário,5.926842,4.879727
Diretoria Regional de Educação - Campo Limpo,5.188091,2.707139
Diretoria Regional de Educação - Pirituba,2.068438,1.698392
Diretoria Regional de Educação - São Mateus,1.941934,1.321618
Diretoria Regional de Educação - Penha,1.840885,1.256791
Diretoria Regional de Educação - Guaianases,2.083498,1.084195
Diretoria Regional de Educação - São Miguel,1.594736,1.003562
Diretoria Regional de Educação - Freguesia/Brasilândia,1.374205,0.829390
Diretoria Regional de Educação - Ipiranga,0.870834,0.772205


,valTotalEmpenhado,valLiquidado
txtGrupoDespesa,,
Outras Despesas Correntes,25.59243,18.071561
Investimentos,1.14000,0.569343


,valTotalEmpenhado,valLiquidado
txtModalidadeAplicacao,,
Aplicações Diretas,26.73243,18.640903


,valTotalEmpenhado,valLiquidado
txtRazaoSocial,,
ELEVADORES ORION LTDA.,7.629500,5.460767
CAMARGO RODRIGUES LTDA,3.039063,2.783454
ELEPAR ELEVADORES LTDA,1.190297,0.666701
CONSTRUTORA PROGREDIOR LTDA,1.140000,0.569343
ACECO TI S.A.,0.583397,0.522691
GOLDEN DISTRIBUIDORA LTDA.,0.552979,0.460816
BOSTON SERVICE DO BRASIL LTDA - EPP,0.432209,0.432209
L & M COMERCIO E SERVIÇOS DE MANUTENÇÃO LTDA - EPP,0.656720,0.395817
EBER QUINTINO GOMES ENGENHARIA - ME,0.471022,0.388383


,txtObjetoContrato
338,SP/MP AQUISIÇÃO DE PEÇAS PARA MANUTENÇÃO DE ELEVADORES DO CEI/EMEI/EMEF E DO BLOCO CULTURAL CEU ...
372,NaN
375,SP/MP - CONTRATAÇÃO DE EMPRESA ESPECIALIZADA NA PRESTAÇÃO DE SERVIÇOS DE MANUTENÇÃO\r\nPREVENTI...
376,SP/MP CONTRATAÇÃO DE EMPRESA PARA SERVIÇOS DE CONSERTO E MANUTENÇÃO COM FORNECIMENTO DE PEÇAS PA...
377,SP/MP - CONTRATAÇÃO PARA SERVIÇOS DE MANUTENÇÃO DE ELÉTRICA NO CEU PARQUE SÃO CARLOS
378,SP/IT - CONTRATAÇÃO PARA MANUTENÇÃO DA PARTE ELÉTRICA NO CEU PARQUE VEREDAS
379,SP/MP - MANUTENÇÃO PREDIAL DE REPARO EM TELHADO E CALHAS DO PRÉDIO DA GESTÃO DO CEU TRÊS PONTES
380,SP/IT CONTRATAÇÃO DE EMPRESA PARA SERVIÇOS DE REPAROS NAS INSTALAÇÕES CENOTÉCNICAS DO TEATRO DO ...
381,SP/MP CONTRATAÇÃO DE EMPRESA PARA SERVIÇOS DE HIDRÁULICA E REMOÇÃO DE TENDA E QUIOSQUES CEU JARD...
382,SP/MP CONTRATAÇÃO DE EMPRESA PARA SERVIÇOS DE HIDRÁULICA E REMOÇÃO DE TENDA E QUIOSQUES CEU VILA...


## 3.6 Aluguel de imóveis

In [116]:
aluguel_imoveis_filter = (empenhos_df['txtDescricaoItemDespesa'].str.contains('Locação de Imóveis', case=False))
aluguel_imoveis = (empenhos_df[aluguel_imoveis_filter])
empenhos_df.loc[aluguel_imoveis_filter, 'cat_1'] = 'funcionamento_unidades'
empenhos_df.loc[aluguel_imoveis_filter, 'cat_2'] = 'aluguel_imoveis'
groupSum(aluguel_imoveis)

,valTotalEmpenhado,valLiquidado
txtCategoriaEconomica,,
Despesas Correntes,22.911279,17.577437


,valTotalEmpenhado,valLiquidado
txtDescricaoElemento,,
Outros Serviços de Terceiros -Pessoa Jurídica,13.693774,10.280610
Outros Serviços de Terceiros - Pessoa Física,9.034796,7.144533
Outros Serviços de Terceiros - Pessoa Jurídica,0.182709,0.152294


,valTotalEmpenhado,valLiquidado
txtDescricaoFonteRecurso,,
Tesouro Municipal,22.911279,17.577437


,valTotalEmpenhado,valLiquidado
txtDescricaoItemDespesa,,
Locação de Imóveis,22.911279,17.577437


,valTotalEmpenhado,valLiquidado
txtDescricaoPrograma,,
Melhoria da qualidade e ampliação do acesso à educação,17.145030,13.578413
Suporte Administrativo,5.766249,3.999024


...

,valTotalEmpenhado,valLiquidado
txtDescricaoUnidade,,
Diretoria Regional de Educação - Santo Amaro,5.186359,4.249820
Diretoria Regional de Educação - Capela do Socorro,4.972329,3.589969
Diretoria Regional de Educação - Ipiranga,3.098174,2.614502
Diretoria Regional de Educação - Butantã,2.261954,1.263452
Diretoria Regional de Educação - São Mateus,0.961079,0.857841
Diretoria Regional de Educação - São Miguel,0.794719,0.771737
Diretoria Regional de Educação - Guaianases,0.767724,0.690185
Gabinete do Secretário,1.175699,0.577142
Diretoria Regional de Educação - Itaquera,0.734170,0.559381


,valTotalEmpenhado,valLiquidado
txtGrupoDespesa,,
Outras Despesas Correntes,22.911279,17.577437


,valTotalEmpenhado,valLiquidado
txtModalidadeAplicacao,,
Aplicações Diretas,22.728570,17.425143
"Apl.Dir.Dec.Op. entre Órgãos,Fundos e Ent.Int.Orç.Fiscal e Seg.Social",0.142468,0.112054
Transferências a Instituições Privadas sem Fins Lucrativos,0.040241,0.040241


,valTotalEmpenhado,valLiquidado
txtRazaoSocial,,
MASTER ADMINISTRACAO DE IMOVEIS PROPRIOS LTDA,2.072770,1.237733
FEMARJAN COSTRUTORA LTDA,1.694046,1.129028
SEARA BENDITA INSTITUICAO ESPIRITA,1.173829,0.876346
MCT ADMINISTRAÇÃO DE IMÓVEIS LTDA - EPP,0.964000,0.746000
ENAL EMPREENDIMENTOS LTDA,1.473781,0.687913
HAGE ADMINISTRAÇÃO DE BENS LTDA.- ME,0.724383,0.657883
ROLEX CONSULTORIA E ADMINISTRAÇÃO DE BENS E NEGÓCIOS LTDA - ME,0.779624,0.649086
S.R. ADMINISTRADORA DE BENS PROPRIOS - EIRELI,0.593349,0.588000
OTTAVIANI EMPREENDIMENTO IMOBILIARIO LTDA,0.601519,0.555336


,txtObjetoContrato
571,NaN


## 3.7 Obras e reformas

In [117]:
obras_reformas_filter = (
    (empenhos_df['txtDescricaoProjetoAtividade'].str.contains('constru', case=False))
    |
    (empenhos_df['txtDescricaoProjetoAtividade'].str.contains('reforma', case=False))
)
obras_reformas = (empenhos_df[obras_reformas_filter])
empenhos_df.loc[obras_reformas_filter, 'cat_1'] = 'funcionamento_unidades'
empenhos_df.loc[obras_reformas_filter, 'cat_2'] = 'obras_reformas'
groupSum(obras_reformas)

,valTotalEmpenhado,valLiquidado
txtCategoriaEconomica,,
Despesas de Capital,109.748075,58.263296


,valTotalEmpenhado,valLiquidado
txtDescricaoElemento,,
Obras e Instalações,89.634945,39.131728
Indenizações e Restituições,14.682730,14.682730
Outros Serviços de Terceiros - Pessoa Jurídica,4.668341,3.686780
Aquisição de Imóveis,0.762059,0.762059


,valTotalEmpenhado,valLiquidado
txtDescricaoFonteRecurso,,
Tesouro Municipal,50.697717,23.415711
Outras Fontes,33.979873,16.566437
Transferências Estaduais,20.010323,16.419021
Transferências Federais,5.060162,1.862127


,valTotalEmpenhado,valLiquidado
txtDescricaoItemDespesa,,
Escolas e Similares,89.634945,39.131728
Restituições a Pessoas Jurídicas,14.682730,14.682730
Gerenciamento,3.528341,3.117437
Desapropriações,0.762059,0.762059
Manutenção e Reformas de Bens Imóveis,1.140000,0.569343


,valTotalEmpenhado,valLiquidado
txtDescricaoPrograma,,
Melhoria da qualidade e ampliação do acesso à educação,109.748075,58.263296


...

,valTotalEmpenhado,valLiquidado
txtDescricaoUnidade,,
Gabinete do Secretário,109.748075,58.263296


,valTotalEmpenhado,valLiquidado
txtGrupoDespesa,,
Investimentos,109.748075,58.263296


,valTotalEmpenhado,valLiquidado
txtModalidadeAplicacao,,
Aplicações Diretas,109.748075,58.263296


,valTotalEmpenhado,valLiquidado
txtRazaoSocial,,
SAO PAULO SECRETARIA DA EDUCACAO,14.682730,14.682730
LOPES KALIL ENGENHARIA E COMERCIO LTDA,24.832149,14.212378
LEMAM CONSTRUCOES E COMERCIO S.A.,15.664034,5.957008
CONSÓRCIO FBS - LOPES KALIL,14.142956,4.945245
CONSTRUMEDICI ENGENHARIA E COMÉRCIO LTDA,5.406643,3.873227
CONSTRUTORA CRONACON LTDA,10.552064,2.944424
FLASA ENGENHARIA E CONSTRUCOES LTDA,6.325143,1.975835
SIMETRICA ENGENHARIA LTDA,2.269715,1.835567
DB CONSTRUCOES LTDA,2.148101,1.447351


,txtObjetoContrato
18355,NaN
18719,"CONFECÇÃO DE COLETES, LONAS E XADREZ"


## 3.8 CEUs - outros custos

In [118]:
ceu_filter = (
    (empenhos_df['txtDescricaoProjetoAtividade'].str.contains('Operação e Manutenção dos Centros Educacionais Unificados', case=False))
    &
    (
        (empenhos_df['txtDescricaoItemDespesa'].str.contains('Outros Serviços de Áudio, Vídeo, Foto, Som, Luz e Imagens', case=False))
        |
        (empenhos_df['txtDescricaoItemDespesa'].str.contains('Lavanderia em Geral', case=False))
        |
        (empenhos_df['txtDescricaoItemDespesa'].str.contains('Monitoramento Aquático', case=False))
    )
)
ceu = (empenhos_df[ceu_filter])
empenhos_df.loc[ceu_filter, 'cat_1'] = 'funcionamento_unidades'
empenhos_df.loc[ceu_filter, 'cat_2'] = 'ceu_outros'
groupSum(ceu)

,valTotalEmpenhado,valLiquidado
txtCategoriaEconomica,,
Despesas Correntes,28.004133,23.396643


,valTotalEmpenhado,valLiquidado
txtDescricaoElemento,,
Outros Serviços de Terceiros -Pessoa Jurídica,28.004133,23.396643


,valTotalEmpenhado,valLiquidado
txtDescricaoFonteRecurso,,
Tesouro Municipal,28.004133,23.396643


,valTotalEmpenhado,valLiquidado
txtDescricaoItemDespesa,,
"Outros Serviços de Áudio, Vídeo, Foto, Som, Luz e Imagens",9.379732,8.203001
Lavanderia em Geral,8.897260,8.152681
Monitoramento Aquático,9.727140,7.040960


,valTotalEmpenhado,valLiquidado
txtDescricaoPrograma,,
Melhoria da qualidade e ampliação do acesso à educação,28.004133,23.396643


...

,valTotalEmpenhado,valLiquidado
txtDescricaoUnidade,,
Gabinete do Secretário,28.004133,23.396643


,valTotalEmpenhado,valLiquidado
txtGrupoDespesa,,
Outras Despesas Correntes,28.004133,23.396643


,valTotalEmpenhado,valLiquidado
txtModalidadeAplicacao,,
Aplicações Diretas,28.004133,23.396643


,valTotalEmpenhado,valLiquidado
txtRazaoSocial,,
MARTINS & LOCOCO LAVANDERIA LTDA,8.897260,8.152681
E.SERVICE COMERCIO E SERVIÇOS LTDA,5.162101,3.413503
FLASH ILUMINACÃO E SONORIZACÃO LTDA - EPP,3.304667,3.016439
LUME SERVIÇOS GERAIS LTDA EPP,2.899012,2.504761
SHAMOU - ESPORTES COMÉRCIO E SERVIÇOS LTDA - ME,2.130006,1.839776
BAKMAR ELETRONICA LTDA ME,1.349765,1.173434
FLYPPER SALVAMENTO AQUÁTICO LTDA. EPP,1.503474,0.950726
FHB - COMERCIAL ELETRÔNICA LTDA ME,1.137263,0.887909
ARCOLIMP SERVIÇOS GERAIS LTDA,0.764375,0.684738


,txtObjetoContrato
7191,NaN
7444,Contratação emergencial de manutenção de Iluminação de 23 CEUs
7445,Contratação emergencial de serviços de manutenção dos equipamentos de Iluminação dos Teatros de ...
7447,Contratação emergencial de serviços de manutenção dos equipamentos de Som dos Teatros de CEUs.
7450,Contratação emergencial de manutenção de Som de 23 CEUs
7451,CONTRATAÇÃO DE AGENTE RECREATIVO - RECREIO NAS FÉRIAS EDIÇÃO JULHO DE 2017
7478,PR-SM(S.MATEUS) – PR-VP/SB (SAPOPEMBA) – CONTRATAÇÃO DE OFICINEIROS - RECREIO NAS FÉRIAS JULHO/2017
7488,SERVIÇOS GERAIS DE MANUTENÇÃO PREDIAL - EMEF. JARDIM PAULO VI


## 3.9 Eventos

In [119]:
# suporte adm - outros
eventos_filter = (
    (empenhos_df['txtDescricaoItemDespesa'].str.contains('Eventos Esportivos, Culturais e Turísticos', case=False))
)
eventos = (empenhos_df[eventos_filter])
empenhos_df.loc[eventos_filter, 'cat_1'] = 'funcionamento_unidades'
empenhos_df.loc[eventos_filter, 'cat_2'] = 'eventos'
groupSum(eventos)

,valTotalEmpenhado,valLiquidado
txtCategoriaEconomica,,
Despesas Correntes,8.018114,6.497739


,valTotalEmpenhado,valLiquidado
txtDescricaoElemento,,
Outros Serviços de Terceiros -Pessoa Jurídica,8.018114,6.497739


,valTotalEmpenhado,valLiquidado
txtDescricaoFonteRecurso,,
Tesouro Municipal,8.018114,6.497739


,valTotalEmpenhado,valLiquidado
txtDescricaoItemDespesa,,
"Eventos Esportivos, Culturais e Turísticos",7.310114,5.812939
"Eventos Esportivos, Culturais e Turísticos - Cooperativas",0.708000,0.684800


,valTotalEmpenhado,valLiquidado
txtDescricaoPrograma,,
Melhoria da qualidade e ampliação do acesso à educação,7.837514,6.392139
Modernização e descentralização da gestão e prestação de serviços,0.151600,0.076600
Suporte Administrativo,0.029000,0.029000


...

,valTotalEmpenhado,valLiquidado
txtDescricaoUnidade,,
Gabinete do Secretário,7.870973,6.350599
Diretoria Regional de Educação - Penha,0.038060,0.038060
Diretoria Regional de Educação - Butantã,0.027095,0.027095
Diretoria Regional de Educação - São Mateus,0.021710,0.021710
Diretoria Regional de Educação - Campo Limpo,0.014595,0.014595
Diretoria Regional de Educação - Guaianases,0.013280,0.013280
Diretoria Regional de Educação - São Miguel,0.012300,0.012300
Diretoria Regional de Educação - Ipiranga,0.010620,0.010620
Diretoria Regional de Educação - Itaquera,0.006240,0.006240


,valTotalEmpenhado,valLiquidado
txtGrupoDespesa,,
Outras Despesas Correntes,8.018114,6.497739


,valTotalEmpenhado,valLiquidado
txtModalidadeAplicacao,,
Aplicações Diretas,8.018114,6.497739


,valTotalEmpenhado,valLiquidado
txtRazaoSocial,,
TOKA EVENTOS ARTISTICOS EIRELI,0.702000,0.636000
COOPERATIVA PAULISTA DE TEATRO,0.564000,0.548800
CONFEDERAÇÃO BRASILEIRA DE ARTES MARCIAIS CHINESAS - KUNG FU,0.630115,0.381328
BAIOQUE PRODUCOES ARTISTICAS LTDA - ME,0.350000,0.350000
FEDERAÇÃO DO DESPORTO ESCOLAR DO ESTADO DE SÃO PAULO,0.330915,0.279311
BOLZAN E ASCHENBACH EVENTOS LTDA,0.276000,0.276000
FEDERAÇÃO PAULISTA DE KARATE,0.258095,0.208461
NAJARA CINE VIDEO E EVENTOS LTDA,0.603800,0.200200
FEDERAÇÃO PAULISTA DE ARTES MARCIAIS INTERESTILOS,0.223940,0.194742


,txtObjetoContrato
682,"AUTORIZO a Contratação de Arbitragem conforme Memorando Inicial Nº SEI 3884008, cuja detentora é..."
4929,NaN
4932,Locação de Brinquedos Infláveis - Recreio nas Férias - Edição Julho/2017.
4933,Apresentações do espetáculo: “Zuando Som Apresenta Faz de Conta”
4934,"Contratação da empresa ""Super Produções Geniais Ltda. ME"", com vistas à realização de 08 (oito) ..."
4935,Apresentações do Espetáculo “QUATRO FACES DO HERÓI”
4936,Apresentações do Espetáculo “CONTOS DE GRIMM: PRINCESAS E BRUXAS”
4937,"Apresentações do Espetáculo ""Números de Circo - Um Espetáculo para Crianças"""
4938,Apresentações do Espetáculo “AVESSO DO AVESSO”
4939,"Apresentações do Espetáculo ""ATÉ AS PRINCESAS SOLTAM PUM"""


## 3.3. Outros?

# 4. Uniformes e materiais escolares

## 4.1. Uniformes

#### Checar filtro: pode estar sub representado

In [120]:
uniformes_filter = empenhos_df['txtDescricaoItemDespesa'].str.contains('uniforme', case=False)
uniformes = (empenhos_df[uniformes_filter])
empenhos_df.loc[uniformes_filter, 'cat_1'] = 'uniforme_materiais'
empenhos_df.loc[uniformes_filter, 'cat_2'] = 'uniforme'
groupSum(uniformes)

,valTotalEmpenhado,valLiquidado
txtCategoriaEconomica,,
Despesas Correntes,300.02518,101.825344


,valTotalEmpenhado,valLiquidado
txtDescricaoElemento,,
"Material, Bem ou Serviço para Distribuição Gratuita",299.956901,101.765057
Outros Serviços de Terceiros -Pessoa Jurídica,0.032845,0.032845
Material de Consumo,0.035434,0.027442


,valTotalEmpenhado,valLiquidado
txtDescricaoFonteRecurso,,
Tesouro Municipal,300.02518,101.825344


,valTotalEmpenhado,valLiquidado
txtDescricaoItemDespesa,,
Uniforme Escolar,299.956901,101.765057
"Uniformes, Camisas, Coletes e Outros",0.032845,0.032845
"Diversos Uniformes, Tecidos e Aviamentos",0.025565,0.017573
Uniformes Esportivos,0.009870,0.009870


,valTotalEmpenhado,valLiquidado
txtDescricaoPrograma,,
Melhoria da qualidade e ampliação do acesso à educação,300.017183,101.817347
Suporte Administrativo,0.007997,0.007997


...

,valTotalEmpenhado,valLiquidado
txtDescricaoUnidade,,
Gabinete do Secretário,299.966473,101.774629
Diretoria Regional de Educação - Penha,0.017870,0.017870
Diretoria Regional de Educação Capela do Socorro,0.015924,0.015924
Diretoria Regional de Educação - Santo Amaro,0.011850,0.011850
Diretoria Regional de Educação - Ipiranga,0.013064,0.005071


,valTotalEmpenhado,valLiquidado
txtGrupoDespesa,,
Outras Despesas Correntes,300.02518,101.825344


,valTotalEmpenhado,valLiquidado
txtModalidadeAplicacao,,
Aplicações Diretas,300.02518,101.825344


,valTotalEmpenhado,valLiquidado
txtRazaoSocial,,
EBN COMERCIO IMPORTACAO E EXPORTACAO S/A,107.089565,36.572437
NILCATEX TEXTIL LTDA,94.554172,24.173376
BRINK MOBIL EQUIPAMENTOS EDUCACIONAIS LTDA,38.947337,19.286998
R.R.INDUSTRIA E COMERCIO DE MALHAS LTDA,19.607706,9.709875
LT GLOBAL COMERCIO E SERVIÇOS EIRELI - ME,18.161577,6.253990
LOTUS COMERCIO LTDA - ME,16.330650,4.312548
BRASILSUL INDUSTRIA E COMERCIO LTDA - EPP,3.799922,1.240188
PLANETA EDUCACIONAL COMERCIO E CONFECCAO LTDA,0.598229,0.215644
SPEZIA COMERCIAL DE MATERIAIS E SUPRIMENTOS EIRELI - ME,0.016258,0.016258


,txtObjetoContrato
6336,CONFECÇÃO DE CAMISETAS PARA O TORNEIO DE XADREZ – DRE IPIRANGA
18786,Aquisição de Kit de Uniforme Escolar
18792,Aquisição de Kit de Uniforme Escolar
18807,AQUISIÇÃO DE KIT UNIFORME ESCOLAR
18815,Aquisição de kits de uniforme escolar lotes 04 e 06
18826,AQUISIÇÃO DE KITS DE UNIFORME ESCOLAR - LOTES 04 E 06
18835,Aquisição de kits de uniforme escolar lote 02
18841,"AQUISIÇÃO DE KITS DE UNIFORME ESCOLAR LOTES 01, 03 E 05"
18842,"Aquisição de Kits de uniforme escolar lotes 01, 03 e 05"
18843,"Aquisição de kits de uniforme escolar lotes 01, 03 e 05"


## 4.2. Materiais

#### (escolares, esportivos, educativos, didáticos, artísticos... como subdividir?)

In [121]:
materiais_filter = (
    (empenhos_df['txtDescricaoItemDespesa'].str.contains('materia', case=False))
    &
    (~empenhos_df['txtDescricaoItemDespesa'].str.contains('comun', case=False))
    &
    (~empenhos_df['txtDescricaoItemDespesa'].str.contains('uniforme', case=False))
)
materiais = (empenhos_df[materiais_filter])
empenhos_df.loc[materiais_filter, 'cat_1'] = 'uniforme_materiais'
empenhos_df.loc[materiais_filter, 'cat_2'] = 'materiais'
groupSum(materiais)

,valTotalEmpenhado,valLiquidado
txtCategoriaEconomica,,
Despesas Correntes,58.755042,26.113732


,valTotalEmpenhado,valLiquidado
txtDescricaoElemento,,
"Material, Bem ou Serviço para Distribuição Gratuita",47.821169,21.005472
Material de Consumo,10.783596,4.957983
Despesas de Exercícios Anteriores,0.150277,0.150277


,valTotalEmpenhado,valLiquidado
txtDescricaoFonteRecurso,,
Tesouro Municipal,58.755042,26.113732


,valTotalEmpenhado,valLiquidado
txtDescricaoItemDespesa,,
Material Escolar,47.743602,20.931055
Materiais Didáticos,2.476033,2.040071
Diversos Materiais de Limpeza e Higienização,1.362784,0.871918
"Diversos Materiais de Cama, Mesa e Banho",0.875004,0.402524
Diversos Materiais de Expediente,0.412113,0.360387
Material Educativo,3.883992,0.302317
Material Esportivo,0.207774,0.155505
Outros Materiais de Consumo,0.150277,0.150277
Materiais para Impressão e Encadernação,0.671882,0.142865


,valTotalEmpenhado,valLiquidado
txtDescricaoPrograma,,
Melhoria da qualidade e ampliação do acesso à educação,57.718734,25.576614
Modernização e descentralização da gestão e prestação de serviços,0.828853,0.371663
Suporte Administrativo,0.206378,0.164377
"Participação, transparência e controle social da administração pública",0.001078,0.001078


...

,valTotalEmpenhado,valLiquidado
txtDescricaoUnidade,,
Gabinete do Secretário,55.147769,23.660507
Diretoria Regional de Educação Capela do Socorro,0.561995,0.542368
Diretoria Regional de Educação - Penha,0.348548,0.345548
Diretoria Regional de Educação - São Miguel,0.557414,0.306702
Diretoria Regional de Educação - Butantã,0.246324,0.236285
Diretoria Regional de Educação - Jaçanã/Tremembé,0.344580,0.232654
Diretoria Regional de Educação - São Mateus,0.258059,0.138259
Diretoria Regional de Educação - Campo Limpo,0.321241,0.133224
Diretoria Regional de Educação - Guaianases,0.196260,0.121262


,valTotalEmpenhado,valLiquidado
txtGrupoDespesa,,
Outras Despesas Correntes,58.755042,26.113732


,valTotalEmpenhado,valLiquidado
txtModalidadeAplicacao,,
Aplicações Diretas,58.755042,26.113732


,valTotalEmpenhado,valLiquidado
txtRazaoSocial,,
BRINK MOBIL EQUIPAMENTOS EDUCACIONAIS LTDA,28.010818,13.565110
ATAKA BRASIL COMERCIO E SERVIÇOS EIRELI,9.892844,4.948793
GRAFICA PRINT INDUSTRIA E EDITORA LTDA,2.779470,1.795257
FUTURA COMERCIO DE MATERIAIS EDUCACIONAIS LTDA - EPP,1.904316,0.927924
THONY PRINT EDITORA GRAFICA LTDA - ME,2.717415,0.898809
ONLINE PAPELARIA E INFORMATICA EIRELI - EPP,1.191670,0.590419
SPEZIA COMERCIAL DE MATERIAIS E SUPRIMENTOS EIRELI - ME,0.540511,0.435086
PRIME STOP COMERCIAL LTDA - ME,0.377421,0.377421
INFO-SIG COMÉRCIO DE SUPRIMENTOS DE INFORMÁTICA EIRELI - ME,0.301925,0.301925


,txtObjetoContrato
341,SP/SR - AQUISIÇÃO DE COPO DESCARTÁVEL PARA ÁGUA PARA ATENDIMENTO DAS NECESSIDADES DAS UNIDADES E...
342,SP/SR AQUISIÇÃO DE 2.000 CENTOS DE CÓPOS DESCARTÁVEIS 150 ML. PARA ÁGUA PARA USO NOS CEU'S
397,SP/SR AQUISIÇÃO DE 32 CILINDROS DE GÁS GLP 45 kG. CADA PARA OS CURSOS DE PANIFICAÇÃO DOS CMCT'S ...
398,SP/SR AQUISIÇÃO DE 40 CILINDROS DE GÁS GLP COM 45 KG. CADA PARA USO NOS CMCT'S - I E II
405,SP/SR AQUISIÇÃO DE 200 CENTOS DE COPOS DESCARTÁVEIS DE 150 ML. PARA ÁGUA PARA USO NO C.M.C.T'S
436,SP/MP AQUISIÇÃO DE MATERIAL DE EXPEDIENTE PARA FORMAÇÃO DE SERVIDORES NA DRE MP
558,SP/MP AQUISIÇÃO DE TINTA PARA IMPRESSORA DE CRACHÁS PARA USO NA DRE-MP
564,SP/MP AQUISIÇÃO DE MATERIAL DE EXPEDIENTE PARA USO NA DRE MP
565,SP/MP AQUISIÇÃO DE 164 PEN DRIVE DE 32 GB PARA USO DA DRE-MP
567,SP/MP AQUISIÇÃO DE 800 CENTOS DE COPOS DESCARTÁVEIS DE 150 ML. PARA ÁGUA PARA SER USADOS NA DRE MP


# 5. Parcerias

In [122]:
parcerias_filter = (
    (empenhos_df['txtDescricaoItemDespesa'].str.contains('convênio', case=False))
    &
    ~(empenhos_df['txtDescricaoItemDespesa'].str.contains('qualificação profissional', case=False))
    &
    ~(empenhos_df['txtDescricaoItemDespesa'].str.contains('conferências', case=False))
)
parcerias = (empenhos_df[parcerias_filter])
empenhos_df.loc[parcerias_filter, 'cat_1'] = 'parcerias'
# empenhos_df.loc[parcerias_filter, 'cat_2'] = ''
groupSum(parcerias)

,valTotalEmpenhado,valLiquidado
txtCategoriaEconomica,,
Despesas Correntes,2292.424092,2004.240927


,valTotalEmpenhado,valLiquidado
txtDescricaoElemento,,
Outros Serviços de Terceiros - Pessoa Jurídica,2292.424092,2004.240927


,valTotalEmpenhado,valLiquidado
txtDescricaoFonteRecurso,,
Tesouro Municipal,2283.978007,2001.647192
Transferências Federais,8.446086,2.593735


,valTotalEmpenhado,valLiquidado
txtDescricaoItemDespesa,,
Convênio - Creche,2230.700574,1955.371328
"Convênios - Alfabetização, Educação e Outros",46.247443,40.399999
Convênios - Portadores de Deficiência,15.476076,8.469600


,valTotalEmpenhado,valLiquidado
txtDescricaoPrograma,,
Melhoria da qualidade e ampliação do acesso à educação,2232.936045,1957.070104
Direitos da pessoa com deficiência,47.550230,36.483657
Superação da extrema pobreza,11.937817,10.687166


...

,valTotalEmpenhado,valLiquidado
txtDescricaoUnidade,,
Diretoria Regional de Educação - Campo Limpo,211.128657,198.508353
Diretoria Regional de Educação - Guaianases,273.353338,195.805506
Diretoria Regional de Educação - São Miguel,191.131776,187.042496
Diretoria Regional de Educação - Pirituba,173.658696,167.766596
Diretoria Regional de Educação - Freguesia/Brasilândia,193.370990,161.123885
Diretoria Regional de Educação - Ipiranga,162.784893,157.419697
Diretoria Regional de Educação - Itaquera,185.470959,156.606269
Diretoria Regional de Educação - Penha,149.866191,142.346145
Diretoria Regional de Educação - São Mateus,148.360836,118.086611


,valTotalEmpenhado,valLiquidado
txtGrupoDespesa,,
Outras Despesas Correntes,2292.424092,2004.240927


,valTotalEmpenhado,valLiquidado
txtModalidadeAplicacao,,
Transferências a Instituições Privadas sem Fins Lucrativos,2292.424092,2004.240927


,valTotalEmpenhado,valLiquidado
txtRazaoSocial,,
SOCIEDADE BENEFICIENTE EQUILIBRIO DE INTERLAGOS,42.737749,36.517114
CRUZADA PRO INFANCIA,33.450280,31.354406
SPDM-ASSOCIAÇÃO PAULISTA PARA O DESENVOLVIMENTO DA MEDICINA,32.364774,28.047264
CENTRO SOCIAL NOSSA SENHORA DO BOM PARTO,28.229937,23.636730
CARITAS DIOCESANA DE CAMPO LIMPO-CDCL,23.977510,21.997971
UNAS - UNIAO DE NUCLEOS ASSOCIACOES DOS MORADORES DE HELIOPOLIS E REGIAO,22.568385,21.621270
INSTITUTO CRIANÇA CIDADÂ,18.914968,17.682746
CENTRO DE ASSISTÊNCIA E PROMOÇÃO SOCIAL NOSSO LAR,20.532360,17.517496
SOCIEDADE BENEFICENTE SÃO CAMILO,20.752715,16.580584


,txtObjetoContrato
439,NaN
444,Aquisição de Certificados Digitais
445,SERVIÇO DE ELÉTRICA - EMEI MARIA DE LOURDES
449,"Programa Movimento de Alfabetização de Jovens e Adultos do município de São Paulo – MOVA / SP, ..."
450,ATENDIMENTO A CRIANÇA POR MEIO DO CENTRO DE EDUCAÇÃO INFANTIL CEI NOVOS RUMOS COM CAPACIDADE CON...
452,Aquisição de passagem aérea
689,CELEBRAÇÃO DE TERMO DE CONVÊNIO - CEI JESUS REI III
693,Repasse de Recursos do PNAE para o CEI MANOEL DE BARROS Cód. INEP 35567115
694,CELEBRAÇÃO DE TERMO DE COLABORAÇÃO - CEI ESTRELA DO MAR
701,CELEBRAÇÃO DE CONVÊNIO - CEI JULIO FRANCISCO DE SOUZA


## 5.1. Convênios com creches

#### Checar filtro

In [123]:
creches_filter = parcerias_filter & (
    (empenhos_df['txtDescricaoProjetoAtividade'].str.contains('Operação e Manutenção de CEIs e Creches da rede conveniada e outras mo', case=False))
)
creches = (empenhos_df[creches_filter])
empenhos_df.loc[creches_filter, 'cat_1'] = 'parcerias'
empenhos_df.loc[creches_filter, 'cat_2'] = 'creches'
groupSum(creches)

,valTotalEmpenhado,valLiquidado
txtCategoriaEconomica,,
Despesas Correntes,2228.235843,1954.064406


,valTotalEmpenhado,valLiquidado
txtDescricaoElemento,,
Outros Serviços de Terceiros - Pessoa Jurídica,2228.235843,1954.064406


,valTotalEmpenhado,valLiquidado
txtDescricaoFonteRecurso,,
Tesouro Municipal,2219.789757,1951.470672
Transferências Federais,8.446086,2.593735


,valTotalEmpenhado,valLiquidado
txtDescricaoItemDespesa,,
Convênio - Creche,2228.235843,1954.064406


,valTotalEmpenhado,valLiquidado
txtDescricaoPrograma,,
Melhoria da qualidade e ampliação do acesso à educação,2228.235843,1954.064406


...

,valTotalEmpenhado,valLiquidado
txtDescricaoUnidade,,
Diretoria Regional de Educação - Campo Limpo,208.947509,196.493778
Diretoria Regional de Educação - Guaianases,272.658228,195.139340
Diretoria Regional de Educação - São Miguel,189.632871,185.791027
Diretoria Regional de Educação - Pirituba,171.742631,165.975216
Diretoria Regional de Educação - Freguesia/Brasilândia,191.832897,159.690220
Diretoria Regional de Educação - Ipiranga,162.083622,156.809580
Diretoria Regional de Educação - Itaquera,184.584608,155.914009
Diretoria Regional de Educação - Penha,149.310823,141.828572
Diretoria Regional de Educação - São Mateus,147.244224,117.174850


,valTotalEmpenhado,valLiquidado
txtGrupoDespesa,,
Outras Despesas Correntes,2228.235843,1954.064406


,valTotalEmpenhado,valLiquidado
txtModalidadeAplicacao,,
Transferências a Instituições Privadas sem Fins Lucrativos,2228.235843,1954.064406


,valTotalEmpenhado,valLiquidado
txtRazaoSocial,,
SOCIEDADE BENEFICIENTE EQUILIBRIO DE INTERLAGOS,42.737749,36.517114
CRUZADA PRO INFANCIA,33.450280,31.354406
CENTRO SOCIAL NOSSA SENHORA DO BOM PARTO,28.229937,23.636730
CARITAS DIOCESANA DE CAMPO LIMPO-CDCL,23.759583,21.781073
UNAS - UNIAO DE NUCLEOS ASSOCIACOES DOS MORADORES DE HELIOPOLIS E REGIAO,22.288221,21.354797
INSTITUTO CRIANÇA CIDADÂ,18.914968,17.682746
CENTRO DE ASSISTÊNCIA E PROMOÇÃO SOCIAL NOSSO LAR,20.532360,17.517496
SOCIEDADE BENEFICENTE SÃO CAMILO,20.622715,16.580584
ASSOCIAÇÃO SÃO SABAS DE FILANTROPIA,15.965346,14.116654


,txtObjetoContrato
689,CELEBRAÇÃO DE TERMO DE CONVÊNIO - CEI JESUS REI III
690,NaN
693,Repasse de Recursos do PNAE para o CEI MANOEL DE BARROS Cód. INEP 35567115
694,CELEBRAÇÃO DE TERMO DE COLABORAÇÃO - CEI ESTRELA DO MAR
701,CELEBRAÇÃO DE CONVÊNIO - CEI JULIO FRANCISCO DE SOUZA
708,"Repasse de recursos para APM de acordo com a Lei Federal nº 9.394/1996, Lei Municipal nº 13.991/..."
711,CEI PEQUENO APRENDIZ
720,CEI FAZENDA DO CARMO
726,CONV. MENSAL CEI DONA MARIA NERI
730,ADITAMENTO PARA ADEQUAÇÃO DE VERBA DE INSTALAÇÃO - CEI LUZIA BEZERRA


## 5.2. Educação especial

In [124]:
ed_especial_filter = parcerias_filter & (
    (empenhos_df['txtDescricaoProjetoAtividade'].str.contains('Convênios para Atendimento de Crianças e Adolescentes com necessidades', case=False))
    |
    (empenhos_df['txtDescricaoProjetoAtividade'].str.contains('Educação Especial', case=False))
)
ed_especial = (empenhos_df[ed_especial_filter])
empenhos_df.loc[ed_especial_filter, 'cat_1'] = 'parcerias'
empenhos_df.loc[ed_especial_filter, 'cat_2'] = 'ed_especial'
groupSum(ed_especial)

,valTotalEmpenhado,valLiquidado
txtCategoriaEconomica,,
Despesas Correntes,47.55023,36.483657


,valTotalEmpenhado,valLiquidado
txtDescricaoElemento,,
Outros Serviços de Terceiros - Pessoa Jurídica,47.55023,36.483657


,valTotalEmpenhado,valLiquidado
txtDescricaoFonteRecurso,,
Tesouro Municipal,47.55023,36.483657


,valTotalEmpenhado,valLiquidado
txtDescricaoItemDespesa,,
"Convênios - Alfabetização, Educação e Outros",32.364774,28.047264
Convênios - Portadores de Deficiência,15.185455,8.436392


,valTotalEmpenhado,valLiquidado
txtDescricaoPrograma,,
Direitos da pessoa com deficiência,47.55023,36.483657


...

,valTotalEmpenhado,valLiquidado
txtDescricaoUnidade,,
Gabinete do Secretário,47.55023,36.483657


,valTotalEmpenhado,valLiquidado
txtGrupoDespesa,,
Outras Despesas Correntes,47.55023,36.483657


,valTotalEmpenhado,valLiquidado
txtModalidadeAplicacao,,
Transferências a Instituições Privadas sem Fins Lucrativos,47.55023,36.483657


,valTotalEmpenhado,valLiquidado
txtRazaoSocial,,
SPDM-ASSOCIAÇÃO PAULISTA PARA O DESENVOLVIMENTO DA MEDICINA,32.364774,28.047264
ASSOCIAÇÃO DE PAIS E AMIGOS DOS EXCEPCIONAIS DE SÃO PAULO,3.939519,1.886311
ASSOCIAÇÃO OBRAS SOCIAIS SANTA CRUZ,0.891831,0.658792
ASSOCIAÇAO NANY APPLE EM BENEFICIO DO OBESO E CARENTE,0.943445,0.638419
AHIMSA ASSOC EDUC PARA MULT DEFIC,0.795604,0.598979
INSTITUTO SEVERINO FABRIANI PARA CRIANÇAS SURDAS,1.735683,0.576806
ASSOCIAÇÃO FILHAS DE SANTA MARIA DA PROVIDENCIA,0.756604,0.519410
ADEFAV - CENTRO DE RECURSOS EM DEFICIENCIA MULTIPLA SURDOCEGUEIRA E DEFICIENCIA VISUAL,0.684021,0.468102
ASSOCIAÇÃO DA CASA DOS DEFICIENTES DE ERMELINO MATARAZZO,0.595999,0.416625


,txtObjetoContrato
53149,NaN
53168,CELEBRAÇÃO DE CONVÊNIO EM CONTINUIDADE - EDUCAÇÃO ESPECIAL INSTITURTO MATER DEI
53171,PAGTO MENSAL - INSTITUIÇÃO BENEFICENTE NOSSO LAR (13/04/2017 À 31/12/2017)
53176,CELEBRAÇÃO DE CONV. EM CONTINUIDADE - E.E. LAR MÃE DO DIVINO AMOR
53178,CELEBRAÇÃO DE CONV. EDUCAÇÃO ESPECIAL - ASSOC. FILHAS DE SANTA MARIA DA PROVIDENCIA
53181,CELEBRAÇÃO DE CONV. EM CONTINUIDADE - ADEFAV (20/04/2017 À 31/12/2017)
53194,CELEBRAÇÃO DE CONVÊNIO EDUC. ESPECIAL - INST. SEVERINO FABRIANI PARA VRIANÇAS SURDAS (29/04/2017...


## 5.3 EJA

In [125]:
eja_filter = parcerias_filter & (
    (empenhos_df['txtDescricaoProjetoAtividade'].str.contains('Alfabetização de Jovens e Adultos', case=False))
)
eja = (empenhos_df[eja_filter])
empenhos_df.loc[eja_filter, 'cat_1'] = 'parcerias'
empenhos_df.loc[eja_filter, 'cat_2'] = 'eja'
groupSum(eja)

,valTotalEmpenhado,valLiquidado
txtCategoriaEconomica,,
Despesas Correntes,11.937817,10.687166


,valTotalEmpenhado,valLiquidado
txtDescricaoElemento,,
Outros Serviços de Terceiros - Pessoa Jurídica,11.937817,10.687166


,valTotalEmpenhado,valLiquidado
txtDescricaoFonteRecurso,,
Tesouro Municipal,11.937817,10.687166


,valTotalEmpenhado,valLiquidado
txtDescricaoItemDespesa,,
"Convênios - Alfabetização, Educação e Outros",11.825031,10.616516
Convênio - Creche,0.112786,0.070650


,valTotalEmpenhado,valLiquidado
txtDescricaoPrograma,,
Superação da extrema pobreza,11.937817,10.687166


...

,valTotalEmpenhado,valLiquidado
txtDescricaoUnidade,,
Diretoria Regional de Educação - Campo Limpo,2.181148,2.014575
Diretoria Regional de Educação - Freguesia/Brasilândia,1.538093,1.433665
Diretoria Regional de Educação - São Miguel,1.354749,1.251469
Diretoria Regional de Educação - São Mateus,1.097055,0.892205
Diretoria Regional de Educação - Santo Amaro,0.776063,0.723247
Diretoria Regional de Educação - Itaquera,0.861663,0.692260
Diretoria Regional de Educação - Guaianases,0.695110,0.666166
Diretoria Regional de Educação - Pirituba,0.727280,0.623614
Diretoria Regional de Educação - Ipiranga,0.633720,0.610117


,valTotalEmpenhado,valLiquidado
txtGrupoDespesa,,
Outras Despesas Correntes,11.937817,10.687166


,valTotalEmpenhado,valLiquidado
txtModalidadeAplicacao,,
Transferências a Instituições Privadas sem Fins Lucrativos,11.937817,10.687166


,valTotalEmpenhado,valLiquidado
txtRazaoSocial,,
ASSOCIAÇÃO EDUCACIONAL E CULTURAL PADRE LUIZ SUTTER,0.720612,0.640699
AVIB - ASSOCIAÇÃO DE VOLUNTÁRIOS INTEGRADOS NO BRASIL,0.425335,0.416667
ASSOCIAÇÃO SÃO JUDAS TADEU E CENTRO COMUNITÁRIO DO JD. IBIRAPUERA,0.420784,0.415589
SOCIEDADE AMIGOS DISTRITO DE PERUS SADIP,0.498359,0.411374
CENTRO DE EDUCAÇÃO DA ZONA LESTE,0.398305,0.386536
INSTITUTO SOCIAL SANTA LUCIA,0.426844,0.353250
UNAS - UNIAO DE NUCLEOS ASSOCIACOES DOS MORADORES DE HELIOPOLIS E REGIAO,0.280163,0.266473
ASSOC A CRIANCA ADOLES JOV. ICARAI,0.270133,0.257146
ASSOCIACAO CANTAREIRA,0.290912,0.249353


,txtObjetoContrato
439,NaN
444,Aquisição de Certificados Digitais
445,SERVIÇO DE ELÉTRICA - EMEI MARIA DE LOURDES
449,"Programa Movimento de Alfabetização de Jovens e Adultos do município de São Paulo – MOVA / SP, ..."
450,ATENDIMENTO A CRIANÇA POR MEIO DO CENTRO DE EDUCAÇÃO INFANTIL CEI NOVOS RUMOS COM CAPACIDADE CON...
452,Aquisição de passagem aérea
12158,CONTRATAÇÃO DE AGENTE RECREATIVO PARA RECREIO NAS FÉRIAS - EDIÇÃO JULHO/2017 - PERIODO DE 10/07 ...
12159,PR-SM(S.MATEUS) – PR-VP/SB (SAPOPEMBA) – CONTRATAÇÃO DE AGENTE DE RECREAÇÃO - RECREIO NAS FÉRIA...
12160,AQUISIÇÃO DE MATERIAL EDUCATIVO - PROGRAMA RECREIO NAS FÉRIAS - JULHO/2017
12161,Aquisição de 140.000 Kg de Extrato de Tomate - ATA de RP n 31/SME/CODAE/2016 - PNAE/FNDE - DECRE...


## 5.3. MOVA

#### Como filtrar?

## 5.4. Outros?

# 6. Obras e reformas

## 6.1. Obras e reformas (talvez devesse passar para a categoria de operação das unidades?)

#### (possível subdividir em unidades educacionais e outros?) 

#### Checar filtro

In [126]:
(empenhos_df[empenhos_df['txtDescricaoProjetoAtividade'].str.contains('reforma', case=False)]
    .txtDescricaoProjetoAtividade.unique())

array(['Construção, reforma e ampliação de equipamentos educacionais',
       'Reformas de Equipamentos Educacionais'], dtype=object)

In [127]:
# obras_reformas_filter = (
#     (empenhos_df['txtDescricaoProjetoAtividade'].str.contains('constru', case=False))
#     |
#     (empenhos_df['txtDescricaoProjetoAtividade'].str.contains('reforma', case=False))
# )
# obras_reformas = (empenhos_df[obras_reformas_filter])
# empenhos_df.loc[obras_reformas_filter, 'cat_1'] = 'obras_reformas'
# empenhos_df.loc[obras_reformas_filter, 'cat_2'] = 'obras_reformas'
# groupSum(obras_reformas)

# 7. Alimentação

#### Checar filtro

In [128]:
(empenhos_df[
    (empenhos_df['txtDescricaoProjetoAtividade'].str.contains('alimenta', case=False))
    |
    (empenhos_df['txtDescricaoProjetoAtividade'].str.contains('leite', case=False))
]
.txtDescricaoProjetoAtividade.unique())

array(['Alimentação Escolar',
       'Programa Nacional de Alimentação Escolar - PNAE/ FNDE',
       'Leve-Leite'], dtype=object)

In [129]:
alimentacao_filter = (
    (empenhos_df['txtDescricaoProjetoAtividade'].str.contains('alimenta', case=False))
    |
    (empenhos_df['txtDescricaoProjetoAtividade'].str.contains('leite', case=False))
)
alimentacao = (empenhos_df[alimentacao_filter])
empenhos_df.loc[alimentacao_filter, 'cat_1'] = 'alimentacao'
# empenhos_df.loc[alimentacao_filter, 'cat_2'] = ''
groupSum(alimentacao)

,valTotalEmpenhado,valLiquidado
txtCategoriaEconomica,,
Despesas Correntes,1026.362386,646.263529


,valTotalEmpenhado,valLiquidado
txtDescricaoElemento,,
Outros Serviços de Terceiros -Pessoa Jurídica,700.795726,392.613835
Material de Consumo,258.694712,194.498042
"Material, Bem ou Serviço para Distribuição Gratuita",42.994545,39.280542
Contribuições,23.877403,19.871110


,valTotalEmpenhado,valLiquidado
txtDescricaoFonteRecurso,,
Transferências Federais,922.274282,581.286497
Tesouro Municipal,104.088104,64.977033


,valTotalEmpenhado,valLiquidado
txtDescricaoItemDespesa,,
Refeições Prontas,593.326962,328.161845
Diversos Gêneros Alimentícios,159.069208,118.266813
Leite e Achocolatados,42.994545,39.280542
Preparo e Distribuição de Alimentos,33.367069,29.668550
Carnes em Geral,30.159054,23.368220
Armazenagem de Alimentos,37.534199,23.237049
"Outras Instituições de Caráter Assistencial, Cultural ou Educacional",23.877403,19.871110
Laticínios em Geral,15.931271,15.732020
"Pães, Doces, Biscoitos e Salgados",20.711276,15.177323


,valTotalEmpenhado,valLiquidado
txtDescricaoPrograma,,
Melhoria da qualidade e ampliação do acesso à educação,1026.362386,646.263529


...

,valTotalEmpenhado,valLiquidado
txtDescricaoUnidade,,
Departamento de Alimentação Escolar,862.147511,553.323166
Coordenadoria de Alimentação Escolar,164.214876,92.940363


,valTotalEmpenhado,valLiquidado
txtGrupoDespesa,,
Outras Despesas Correntes,1026.362386,646.263529


,valTotalEmpenhado,valLiquidado
txtModalidadeAplicacao,,
Aplicações Diretas,1002.484983,626.392419
Transferências a Instituições Privadas sem Fins Lucrativos,23.877403,19.871110


,valTotalEmpenhado,valLiquidado
txtRazaoSocial,,
SHA COMERCIO DE ALIMENTOS LTDA,190.741737,118.178282
DENJUD REFEIÇÕES COLETIVAS ADMINISTRAÇÃO E SERVIÇOS LTDA,144.606175,77.517622
COMERCIAL MILANO BRASIL LTDA,92.039220,74.653773
APETECE SISTEMAS DE ALIMENTACAO S/A,161.392758,73.424133
TANGARÁ IMP.E EXP.S/A,61.845260,55.286151
BASE SISTEMA SERVIÇOS DE ADMINISTRAÇÃO E COMERCIO LTDA,65.781097,40.209289
P.R.M. SERVIÇOS E MAO DE OBRA ESPECIALIZADA EIRELI,44.011233,34.421740
SERBOM ARMAZENS GERAIS FRIGORIFICOS LTDA,37.534199,23.237049
CASTOR ALIMENTOS LTDA,22.742754,19.027426


,txtObjetoContrato
1000,"Serviços de Nutrição e Alimentação Escolar - Regiões de GUAIANASES, ITAQUERA e PENHA \n\n"
1001,Serviços de Nutrição e Alimentação Escolar - Região da FREGUESIA DO Ó e SÃO MATEUS
1003,CONSUMO DE ÁGUA E SERVIÇO DE ÁGUA E ESGOTO-UNIDADES ESCOLARES E DRESA
1006,MANUTENÇÃO PREDIAL - CEI VILA LIBANESA ELETRICA
1007,Contratação de serviços de nutrição e alimentação escola - Lote 4 - FREGUESIA DO O.
1008,Contratação de serviços de nutrição e alimentação escolar – LOTE 7B - IPIRANGA II.\r\n\r\n
1009,Contratação de serviços de nutrição e alimentação escolar - Lote 5 - FREGUESIA DO O.
1010,Contratação de seviços e nutrição e alimentação escolar - LOTE 7B - IPIRANGA II.
1011,Contratação de serviços de nutrição e alimentação escolar – LOTE 9 - JAÇANA/TREMEMBÉ
1012,Contratação de serviços de nutrição e alimentação escolar - Lote 7A - IPIRANGA I.


## 7.1. Refeições prontas

#### Checar filtro e filtro anterior

In [130]:
refeicoes_filter = alimentacao_filter & (
    (empenhos_df['txtDescricaoItemDespesa'].str.contains('Refeições Prontas', case=False))
    &
    ~(empenhos_df['txtDescricaoProjetoAtividade'].str.contains('leite', case=False))
)
refeicoes = (empenhos_df[refeicoes_filter])
empenhos_df.loc[refeicoes_filter, 'cat_1'] = 'alimentacao'
empenhos_df.loc[refeicoes_filter, 'cat_2'] = 'refeicoes'
groupSum(refeicoes)

,valTotalEmpenhado,valLiquidado
txtCategoriaEconomica,,
Despesas Correntes,593.326962,328.161845


,valTotalEmpenhado,valLiquidado
txtDescricaoElemento,,
Outros Serviços de Terceiros -Pessoa Jurídica,593.326962,328.161845


,valTotalEmpenhado,valLiquidado
txtDescricaoFonteRecurso,,
Transferências Federais,593.326962,328.161845


,valTotalEmpenhado,valLiquidado
txtDescricaoItemDespesa,,
Refeições Prontas,593.326962,328.161845


,valTotalEmpenhado,valLiquidado
txtDescricaoPrograma,,
Melhoria da qualidade e ampliação do acesso à educação,593.326962,328.161845


...

,valTotalEmpenhado,valLiquidado
txtDescricaoUnidade,,
Departamento de Alimentação Escolar,586.456048,326.603238
Coordenadoria de Alimentação Escolar,6.870914,1.558606


,valTotalEmpenhado,valLiquidado
txtGrupoDespesa,,
Outras Despesas Correntes,593.326962,328.161845


,valTotalEmpenhado,valLiquidado
txtModalidadeAplicacao,,
Aplicações Diretas,593.326962,328.161845


,valTotalEmpenhado,valLiquidado
txtRazaoSocial,,
SHA COMERCIO DE ALIMENTOS LTDA,163.581916,93.378398
DENJUD REFEIÇÕES COLETIVAS ADMINISTRAÇÃO E SERVIÇOS LTDA,143.268568,76.278551
APETECE SISTEMAS DE ALIMENTACAO S/A,161.392758,73.424133
BASE SISTEMA SERVIÇOS DE ADMINISTRAÇÃO E COMERCIO LTDA,60.911457,36.579694
P.R.M. SERVIÇOS E MAO DE OBRA ESPECIALIZADA EIRELI,44.011233,34.421740
COMERCIAL MILANO BRASIL LTDA,11.177524,7.877897
AEX ALIMENTA COMERCIO DE REFEIÇÕES E SERVIÇOS LTDA,8.983506,6.201431


,txtObjetoContrato
1000,"Serviços de Nutrição e Alimentação Escolar - Regiões de GUAIANASES, ITAQUERA e PENHA \n\n"
1001,Serviços de Nutrição e Alimentação Escolar - Região da FREGUESIA DO Ó e SÃO MATEUS
1003,CONSUMO DE ÁGUA E SERVIÇO DE ÁGUA E ESGOTO-UNIDADES ESCOLARES E DRESA
1006,MANUTENÇÃO PREDIAL - CEI VILA LIBANESA ELETRICA
1007,Contratação de serviços de nutrição e alimentação escola - Lote 4 - FREGUESIA DO O.
1008,Contratação de serviços de nutrição e alimentação escolar – LOTE 7B - IPIRANGA II.\r\n\r\n
1009,Contratação de serviços de nutrição e alimentação escolar - Lote 5 - FREGUESIA DO O.
1010,Contratação de seviços e nutrição e alimentação escolar - LOTE 7B - IPIRANGA II.
1011,Contratação de serviços de nutrição e alimentação escolar – LOTE 9 - JAÇANA/TREMEMBÉ
1012,Contratação de serviços de nutrição e alimentação escolar - Lote 7A - IPIRANGA I.


## 7.2. Leve-Leite

In [131]:
leve_leite_filter = alimentacao_filter & (
    (empenhos_df['txtDescricaoProjetoAtividade'].str.contains('leite', case=False))
)
leve_leite = (empenhos_df[leve_leite_filter])
empenhos_df.loc[leve_leite_filter, 'cat_1'] = 'alimentacao'
empenhos_df.loc[leve_leite_filter, 'cat_2'] = 'leve_leite'
groupSum(leve_leite)

,valTotalEmpenhado,valLiquidado
txtCategoriaEconomica,,
Despesas Correntes,79.562042,50.826933


,valTotalEmpenhado,valLiquidado
txtDescricaoElemento,,
"Material, Bem ou Serviço para Distribuição Gratuita",42.994545,39.280542
Outros Serviços de Terceiros -Pessoa Jurídica,36.567497,11.546391


,valTotalEmpenhado,valLiquidado
txtDescricaoFonteRecurso,,
Tesouro Municipal,79.562042,50.826933


,valTotalEmpenhado,valLiquidado
txtDescricaoItemDespesa,,
Leite e Achocolatados,42.994545,39.280542
Transporte e Remessa de Encomendas,36.567497,11.546391


,valTotalEmpenhado,valLiquidado
txtDescricaoPrograma,,
Melhoria da qualidade e ampliação do acesso à educação,79.562042,50.826933


...

,valTotalEmpenhado,valLiquidado
txtDescricaoUnidade,,
Departamento de Alimentação Escolar,61.083896,37.872368
Coordenadoria de Alimentação Escolar,18.478146,12.954565


,valTotalEmpenhado,valLiquidado
txtGrupoDespesa,,
Outras Despesas Correntes,79.562042,50.826933


,valTotalEmpenhado,valLiquidado
txtModalidadeAplicacao,,
Aplicações Diretas,79.562042,50.826933


,valTotalEmpenhado,valLiquidado
txtRazaoSocial,,
TANGARÁ IMP.E EXP.S/A,35.921819,32.575381
EMPRESA BRASILEIRA DE CORREIOS E TELÉGRAFOS,36.567497,11.546391
BONISSIMA COMERCIO E SERVIÇOS LTDA. EPP,7.072726,6.705161


,txtObjetoContrato
9042,Aquisição de 224.493 Kg de Leite em Pó Integral - Lote 10 - PROGRAMA LEVE-LEITE - ATA de RP nº 0...
9043,Aquisição 63.138 Kg de Leite em Pó Integral - Lote 7 - PROGRAMA LEVE-LEITE - ATA de RP 09/SME/CO...
9044,Aquisição de 80.677 Kg de Leite em Pó Integral - Lote 9 - PROGRAMA LEVE-LEITE - ATA de RP nº 09/...
9045,Aquisição de 196.431Kg de Leite em Pó Integral – Lote 5 e 6 - PROGRAMA LEVE-LEITE - ATA DE RP N...
9046,Aquisição de 70.154 Kg de Leite em Pó Integral - PROGRAMA LEVE-LEITE - Lote 1 e 2 - ATA DE RP ...
9047,Aquisição de 189.415 Kg de leite em Pó Integral - Lote 8 - PROGRAMA LEVE-LEITE - ATA de RP 07/SM...
9048,Aquisição 87.692 Kg de Leite em Pó Integral - Lote 4 - PROGRAMA LEVE-LEITE - ATA de RP nº 07/SME...
9049,Aquisição 36.545 kg de Leite em Pó Integral – PROGRAMA LEVE-LEITE – referente a 75% da necessid...
9050,Aquisição 146.178 kg de Leite em Pó Integral – PROGRAMA LEVE-LEITE – Lotes 1 e 2 - ATA de RP 07/...
9051,Aquisição 131.560 kg de Leite em Pó Integral – PROGRAMA LEVE-LEITE – Lote 7 - ATA de RP 09/SME/C...


## 7.3. Preparo, distribuição e armazenagem

#### Checar filtro alimentação

In [132]:
prep_dist_filter = alimentacao_filter & (
    (
        (empenhos_df['txtDescricaoItemDespesa'].str.contains('Preparo e Distribuição de Alimentos', case=False))
        |
        (empenhos_df['txtDescricaoItemDespesa'].str.contains('Armazenagem de Alimentos', case=False))
    )
    &
    ~(empenhos_df['txtDescricaoProjetoAtividade'].str.contains('leite', case=False))
)
prep_dist = (empenhos_df[prep_dist_filter])
empenhos_df.loc[prep_dist_filter, 'cat_1'] = 'alimentacao'
empenhos_df.loc[prep_dist_filter, 'cat_2'] = 'prep_dist'
groupSum(prep_dist)

,valTotalEmpenhado,valLiquidado
txtCategoriaEconomica,,
Despesas Correntes,70.901268,52.905599


,valTotalEmpenhado,valLiquidado
txtDescricaoElemento,,
Outros Serviços de Terceiros -Pessoa Jurídica,70.901268,52.905599


,valTotalEmpenhado,valLiquidado
txtDescricaoFonteRecurso,,
Transferências Federais,70.901268,52.905599


,valTotalEmpenhado,valLiquidado
txtDescricaoItemDespesa,,
Preparo e Distribuição de Alimentos,33.367069,29.668550
Armazenagem de Alimentos,37.534199,23.237049


,valTotalEmpenhado,valLiquidado
txtDescricaoPrograma,,
Melhoria da qualidade e ampliação do acesso à educação,70.901268,52.905599


...

,valTotalEmpenhado,valLiquidado
txtDescricaoUnidade,,
Departamento de Alimentação Escolar,58.967661,47.529303
Coordenadoria de Alimentação Escolar,11.933606,5.376297


,valTotalEmpenhado,valLiquidado
txtGrupoDespesa,,
Outras Despesas Correntes,70.901268,52.905599


,valTotalEmpenhado,valLiquidado
txtModalidadeAplicacao,,
Aplicações Diretas,70.901268,52.905599


,valTotalEmpenhado,valLiquidado
txtRazaoSocial,,
SHA COMERCIO DE ALIMENTOS LTDA,27.159821,24.799884
SERBOM ARMAZENS GERAIS FRIGORIFICOS LTDA,37.534199,23.237049
BASE SISTEMA SERVIÇOS DE ADMINISTRAÇÃO E COMERCIO LTDA,4.869640,3.629595
DENJUD REFEIÇÕES COLETIVAS ADMINISTRAÇÃO E SERVIÇOS LTDA,1.337607,1.239071


,txtObjetoContrato
9094,NaN
9113,PAGAMENTO DE SERVIÇO DE FORNECIMENTO DE ÁGUA E ESGOTO - SABESP
9115,CONSUMO DE ÁGUA E SERVIÇO DE ÁGUA E ESGOTO-UNIDADES ESCOLARES E DRESA


## 7.4. Insumos 

#### (para refeições preparadas pela própria SME? Como subdividir?)

#### Checar filtro alimentação e filtros anteriores

In [133]:
insumos_regex = 'Diversos Gêneros Alimentícios|Carnes em Geral|Laticínios em Geral|Pães, Doces, Biscoitos e Salgados|Grãos em Geral|Frutas Naturais|Leite em Pó ou Natural|Óleos, Gorduras e Azeites Comestíveis, Condimentos e Temperos em Geral|Massas Alimentícias|Bebidas em Geral|Farináceos'
insumos_filter = alimentacao_filter & (
    (
        (empenhos_df['txtDescricaoItemDespesa'].str.contains(insumos_regex, case=False))
    )
    &
    ~(empenhos_df['txtDescricaoProjetoAtividade'].str.contains('leite', case=False))
)
insumos = (empenhos_df[insumos_filter])
empenhos_df.loc[insumos_filter, 'cat_1'] = 'alimentacao'
empenhos_df.loc[insumos_filter, 'cat_2'] = 'insumos'
groupSum(insumos)

,valTotalEmpenhado,valLiquidado
txtCategoriaEconomica,,
Despesas Correntes,258.694712,194.498042


,valTotalEmpenhado,valLiquidado
txtDescricaoElemento,,
Material de Consumo,258.694712,194.498042


,valTotalEmpenhado,valLiquidado
txtDescricaoFonteRecurso,,
Transferências Federais,234.168650,180.347942
Tesouro Municipal,24.526063,14.150100


,valTotalEmpenhado,valLiquidado
txtDescricaoItemDespesa,,
Diversos Gêneros Alimentícios,159.069208,118.266813
Carnes em Geral,30.159054,23.368220
Laticínios em Geral,15.931271,15.732020
"Pães, Doces, Biscoitos e Salgados",20.711276,15.177323
Grãos em Geral,12.048248,9.668589
Frutas Naturais,10.804552,4.785197
Leite em Pó ou Natural,2.847350,2.847350
"Óleos, Gorduras e Azeites Comestíveis, Condimentos e Temperos em Geral",4.226210,1.855835
Massas Alimentícias,1.560750,1.459940


,valTotalEmpenhado,valLiquidado
txtDescricaoPrograma,,
Melhoria da qualidade e ampliação do acesso à educação,258.694712,194.498042


...

,valTotalEmpenhado,valLiquidado
txtDescricaoUnidade,,
Departamento de Alimentação Escolar,143.758840,133.062779
Coordenadoria de Alimentação Escolar,114.935873,61.435264


,valTotalEmpenhado,valLiquidado
txtGrupoDespesa,,
Outras Despesas Correntes,258.694712,194.498042


,valTotalEmpenhado,valLiquidado
txtModalidadeAplicacao,,
Aplicações Diretas,258.694712,194.498042


,valTotalEmpenhado,valLiquidado
txtRazaoSocial,,
COMERCIAL MILANO BRASIL LTDA,80.861696,66.775876
TANGARÁ IMP.E EXP.S/A,25.923441,22.710770
CASTOR ALIMENTOS LTDA,22.742754,19.027426
ATV ASSESSORIA TÉCNICA EM VENDAS LTDA,17.002894,12.596724
PRO ATIVA ALIMENTOS LTDA - EPP,14.265284,11.399347
FRESKITO PRODUTOS ALIMENTICIOS LTDA,17.755644,11.225533
MULTICOM COMERCIO MULTIPLO DE ALIMENTOS LTDA,9.510698,7.138311
GONÇALVES & TORTOLA S/A,4.788000,4.787395
VIGOR ALIMENTOS /A,3.087080,2.948434


,txtObjetoContrato
8918,NaN
8920,Aquisição de 380.000 Kg de Cortes Congelados de Frango – filé de coxa e sobrecoxa sem pele – Ata...
8924,Aquisição de 160.000 Kg de Carne Bovina Congelada em Peças Cortada ao meio Corte Coxão Duro - d...
8925,Aquisição de 219.375 Kg de Carne Bovina Congelada em Peça - Corte Coxão Duro - ATA de RP nº 022/...
8926,Aquisição de 139.000 Kg de Carne Suina - AF - Chamada Pública nº 010/SME/CODAE/2017 - FNDE/PNAE.
8927,Aquisição de 140.000 Kg de Achocolatado em Pó - ATA DE RP Nº 016/SME/CODAE/2017 - FNDE/PNAE.
8928,Aquisição de 73.125 Kg de Carne Bovina Congelada em Peças - Corte Coxão Duro - ATA de RP nº 021/...
8929,Aquisição de 435.000 Kg de Carne Bovina congelada em peças cortadas ao meio corte Coxão Duro - ...
8930,116.700 Kg de Pão Tipo Bisnaguinha Tradicional e 36.900 Kg de Pão Tipo Bisnaguinha Integral - AT...
8934,Aquisição de 70.400 Kg de Pão Tipo Hot Dog Integral - ATA de RP 12/SME/CODAE//2016.


## 7.5. Transferencias PNAE

In [134]:
pnae_filter = alimentacao_filter & (
    (
        (empenhos_df['txtDescricaoProjetoAtividade'].str.contains('Programa Nacional de Alimentação Escolar - PNAE/ FNDE', case=False))
        &
        (empenhos_df['txtDescricaoItemDespesa'].str.contains('Outras instituições', case=False))
    )
    &
    ~(empenhos_df['txtDescricaoProjetoAtividade'].str.contains('leite', case=False))
)
pnae = (empenhos_df[pnae_filter])
empenhos_df.loc[pnae_filter, 'cat_1'] = 'alimentacao'
empenhos_df.loc[pnae_filter, 'cat_2'] = 'pnae'
groupSum(pnae)

,valTotalEmpenhado,valLiquidado
txtCategoriaEconomica,,
Despesas Correntes,23.877403,19.87111


,valTotalEmpenhado,valLiquidado
txtDescricaoElemento,,
Contribuições,23.877403,19.87111


,valTotalEmpenhado,valLiquidado
txtDescricaoFonteRecurso,,
Transferências Federais,23.877403,19.87111


,valTotalEmpenhado,valLiquidado
txtDescricaoItemDespesa,,
"Outras Instituições de Caráter Assistencial, Cultural ou Educacional",23.877403,19.87111


,valTotalEmpenhado,valLiquidado
txtDescricaoPrograma,,
Melhoria da qualidade e ampliação do acesso à educação,23.877403,19.87111


...

,valTotalEmpenhado,valLiquidado
txtDescricaoUnidade,,
Coordenadoria de Alimentação Escolar,11.996337,11.615632
Departamento de Alimentação Escolar,11.881066,8.255478


,valTotalEmpenhado,valLiquidado
txtGrupoDespesa,,
Outras Despesas Correntes,23.877403,19.87111


,valTotalEmpenhado,valLiquidado
txtModalidadeAplicacao,,
Transferências a Instituições Privadas sem Fins Lucrativos,23.877403,19.87111


,valTotalEmpenhado,valLiquidado
txtRazaoSocial,,
SOCIEDADE BENEFICENTE EQUILIBRIO DE INTERLAGOS,0.940530,0.505682
CENTRO DE ASSISTÊNCIA E PROMOÇÃO SOCIAL NOSSO LAR,0.496266,0.496266
CRUZADA PRÓ INFÂNCIA,0.460956,0.460956
CENTRO SOCIAL NOSSA SENHORA DO BOM PARTO,0.401250,0.401250
AÇÃO SOCIAL COMUNITÁRIA DO LAGEADO JOILSON DE JESUS,0.279270,0.279270
INSTITUTO CEM POR CENTO SOCIAL,0.222346,0.222346
FUNDAÇÃO COMUNIDADE DA GRAÇA,0.219564,0.219564
ASSOCIAÇÃO SANTA EDWIGES,0.185966,0.185966
OBRA ASSISTENCIAL JESUS MENINO,0.185324,0.185324


,txtObjetoContrato
8166,Repasse de Recursos do PNAE para o CEI Alerta Criança - Cód. INEP: 35584824
8167,REPASSE DE RECURSOS DO PNAE PARA O CEI CONVENIADO CENTRO SOCIAL LAUZANE PAULISTA - COD.INEP 3518...
8168,Repasse de Recursos do PNAE para o CEI TIO NINO - CÓD. INEP 35564941\n Repasse de Recursos do ...
8169,Repasse de Recursos do PNAE para o CEI Wenceslau Guimarães/CEI Mooca/CEI Nosso Lar II/CEI Tatua...
8170,Repasse de Recursos do PNAE para o CEI PROFESSORA FLORA - CÓD. INEP 35354454\r\n
8171,Repasse de Recursos do PNAE para o CEI Belém - Cód. INEP: 35182357
8172,Repasse de Recursos do PNAE para o CEI Estrela Nova- CÓD. INEP 35183519
8173,Repasse de Recursos do PNAE para o CEI JULIO FRANCISCO DE SOUZA - CÓD. INEP 35411152\r\n\r\n
8174,Repasse de Recursos do PNAE para o CEI VEREADOR SHIRO KYONO - CÓD. INEP 35182217\r\n
8175,Repasse de Recursos do PNAE para o CEI CASINHA DA VOVÓ - CÓD. INEP 35432921\r\n


# 8. Pedagógico e formação

## 8.1. Cursos e formação

## 8.2. Elaboração de materiais didáticos

## 8.3. Outros?

# 9. Tecnologia da informação

#### Checar filtros

In [135]:
ti_filter = (
    (empenhos_df['txtDescricaoSubFuncao'].str.contains('tecnologia da inf', case=False))
)
ti = (empenhos_df[ti_filter])
empenhos_df.loc[ti_filter, 'cat_1'] = 'ti'
groupSum(ti)

,valTotalEmpenhado,valLiquidado
txtCategoriaEconomica,,
Despesas Correntes,66.708423,40.613301
Despesas de Capital,41.379460,0.284038


,valTotalEmpenhado,valLiquidado
txtDescricaoElemento,,
Outros Serviços de Terceiros -Pessoa Jurídica,66.705633,40.610511
Outros Serviços de Terceiros - Pessoa Jurídica,0.360000,0.180000
Equipamentos e Material Permanente,41.019460,0.104038
Material de Consumo,0.002790,0.002790


,valTotalEmpenhado,valLiquidado
txtDescricaoFonteRecurso,,
Tesouro Municipal,108.087883,40.89734


,valTotalEmpenhado,valLiquidado
txtDescricaoItemDespesa,,
"Serviço de Instalação, Ampliação de Sist. de Telefonia/Teleinformática",15.368131,11.271799
PRODAM,15.912125,9.824414
"Outros Serviços de Apoio Administrativo, Técnico e Operacional",9.316095,6.581965
Manutenção e Conservação de Equipamentos de Processamento de Dados,9.195115,5.937811
Manutenção de Software,8.605008,3.876742
"Supervisão, Acompanhamento e Suporte de Serviços Técnicos",2.559587,1.771597
Equipamentos de Processamento de Dados,0.583397,0.522691
Outras Máquinas e Equipamentos,0.552979,0.460816
Guarda e Gerenciamento de Documentos,4.603634,0.361129


,valTotalEmpenhado,valLiquidado
txtDescricaoPrograma,,
Modernização e descentralização da gestão e prestação de serviços,108.087883,40.89734


...

,valTotalEmpenhado,valLiquidado
txtDescricaoUnidade,,
Gabinete do Secretário,108.087883,40.89734


,valTotalEmpenhado,valLiquidado
txtGrupoDespesa,,
Outras Despesas Correntes,66.708423,40.613301
Investimentos,41.379460,0.284038


,valTotalEmpenhado,valLiquidado
txtModalidadeAplicacao,,
Aplicações Diretas,108.087883,40.89734


,valTotalEmpenhado,valLiquidado
txtRazaoSocial,,
PRIMESYS SOLUÇÕES EMPRESARIAIS S.A.,14.698120,10.708426
EMPRESA DE TECNOLOGIA DA INFORMAÇÃO E COMUNICAÇÃO DO MUNICÍPIO DE SÃO PAULO - PRODAM-SP S/A,15.912125,9.824414
GOLDEN DISTRIBUIDORA LTDA.,9.334915,6.538447
G&P PROJETOS E SISTEMAS LTDA,9.320045,5.895701
MSTECH EDUCACAO E TECNOLOGIA EIRELI,7.853286,3.335047
AYNIL SOLUÇÕES S/A,2.634882,1.842409
TELEFÔNICA BRASIL S.A.,0.670011,0.563374
ACECO TI S.A.,0.583397,0.522691
GARTNER DO BRASIL SERVIÇOS DE PESQUISAS LTDA.,0.534159,0.504334


,txtObjetoContrato
7789,PRESTAÇÃO DE SERVIÇOS DE AGENTE DE RECREAÇÃO NO PROGRAMA RECREIO NAS FÉRIAS - JANEIRO/2017
7791,Cartuchos de tinta para a impressora jato de tinta CANON iP 7210.
7792,\r\n\r\nAquisição de cartuchos de tinta para a impressota jato de tinta CANON iP 7210\r\n
7793,NaN
7797,Serviço de suporte técnico e manutenção p/hardware
7798,"Contratação Emergencial - Suporte Técnico e Manutenção ""On Site"""
7803,REPASSE DE RECURSOS A APM - PTRF: CUSTEIO E CAPITAL / 2017
7819,Aquisição de Certificados Digitais
7820,"Repasse de recursos para APM de acordo com a Lei Federal nº 9.394/1996, Lei Municipal nº 13.991/..."
7824,Repasse de Recursos do PNAE para o CEI MORRO GRANDE CT VIDA - CÓD. INEP 35403118\n


## 9.1. Fornecimento de internet

#### Checar filtro

In [136]:
internet_filter = ti_filter & (
    (empenhos_df['txtDescricaoItemDespesa'].str.contains('Serviço de Instalação, Ampliação de Sist. de Telefonia/Teleinformática', case=False))
)
internet = (empenhos_df[internet_filter])
empenhos_df.loc[internet_filter, 'cat_1'] = 'ti'
empenhos_df.loc[internet_filter, 'cat_2'] = 'internet'
groupSum(internet)

,valTotalEmpenhado,valLiquidado
txtCategoriaEconomica,,
Despesas Correntes,15.368131,11.271799


,valTotalEmpenhado,valLiquidado
txtDescricaoElemento,,
Outros Serviços de Terceiros -Pessoa Jurídica,15.368131,11.271799


,valTotalEmpenhado,valLiquidado
txtDescricaoFonteRecurso,,
Tesouro Municipal,15.368131,11.271799


,valTotalEmpenhado,valLiquidado
txtDescricaoItemDespesa,,
"Serviço de Instalação, Ampliação de Sist. de Telefonia/Teleinformática",15.368131,11.271799


,valTotalEmpenhado,valLiquidado
txtDescricaoPrograma,,
Modernização e descentralização da gestão e prestação de serviços,15.368131,11.271799


...

,valTotalEmpenhado,valLiquidado
txtDescricaoUnidade,,
Gabinete do Secretário,15.368131,11.271799


,valTotalEmpenhado,valLiquidado
txtGrupoDespesa,,
Outras Despesas Correntes,15.368131,11.271799


,valTotalEmpenhado,valLiquidado
txtModalidadeAplicacao,,
Aplicações Diretas,15.368131,11.271799


,valTotalEmpenhado,valLiquidado
txtRazaoSocial,,
PRIMESYS SOLUÇÕES EMPRESARIAIS S.A.,14.698120,10.708426
TELEFÔNICA BRASIL S.A.,0.670011,0.563374


,txtObjetoContrato
7815,NaN


## 9.2. Manutenção de equipamentos

In [137]:
manutencao_equip_ti_filter = ti_filter & (
    (empenhos_df['txtDescricaoItemDespesa'].str.contains('Manutenção e Conservação de Equipamentos de Processamento de Dados', case=False))
)
manutencao_equip_ti = (empenhos_df[manutencao_equip_ti_filter])
empenhos_df.loc[manutencao_equip_ti_filter, 'cat_1'] = 'ti'
empenhos_df.loc[manutencao_equip_ti_filter, 'cat_2'] = 'manutencao_equip_ti'
groupSum(manutencao_equip_ti)

,valTotalEmpenhado,valLiquidado
txtCategoriaEconomica,,
Despesas Correntes,9.195115,5.937811


,valTotalEmpenhado,valLiquidado
txtDescricaoElemento,,
Outros Serviços de Terceiros -Pessoa Jurídica,9.195115,5.937811


,valTotalEmpenhado,valLiquidado
txtDescricaoFonteRecurso,,
Tesouro Municipal,9.195115,5.937811


,valTotalEmpenhado,valLiquidado
txtDescricaoItemDespesa,,
Manutenção e Conservação de Equipamentos de Processamento de Dados,9.195115,5.937811


,valTotalEmpenhado,valLiquidado
txtDescricaoPrograma,,
Modernização e descentralização da gestão e prestação de serviços,9.195115,5.937811


...

,valTotalEmpenhado,valLiquidado
txtDescricaoUnidade,,
Gabinete do Secretário,9.195115,5.937811


,valTotalEmpenhado,valLiquidado
txtGrupoDespesa,,
Outras Despesas Correntes,9.195115,5.937811


,valTotalEmpenhado,valLiquidado
txtModalidadeAplicacao,,
Aplicações Diretas,9.195115,5.937811


,valTotalEmpenhado,valLiquidado
txtRazaoSocial,,
G&P PROJETOS E SISTEMAS LTDA,8.666993,5.445367
COMPWIRE INFORMATICA S/A,0.528122,0.492444


,txtObjetoContrato
7803,REPASSE DE RECURSOS A APM - PTRF: CUSTEIO E CAPITAL / 2017
7804,NaN
7808,Serviço de suporte técnico e manutenção p/hardware
7809,"Contratação Emergencial - Suporte Técnico e Manutenção ""On Site"""


## 9.3. PRODAM

In [138]:
prodam_filter = ti_filter & (
    (empenhos_df['txtDescricaoItemDespesa'].str.contains('PRODAM', case=False))
)
prodam = (empenhos_df[prodam_filter])
empenhos_df.loc[prodam_filter, 'cat_1'] = 'ti'
empenhos_df.loc[prodam_filter, 'cat_2'] = 'prodam'
groupSum(prodam)

,valTotalEmpenhado,valLiquidado
txtCategoriaEconomica,,
Despesas Correntes,15.912125,9.824414


,valTotalEmpenhado,valLiquidado
txtDescricaoElemento,,
Outros Serviços de Terceiros -Pessoa Jurídica,15.912125,9.824414


,valTotalEmpenhado,valLiquidado
txtDescricaoFonteRecurso,,
Tesouro Municipal,15.912125,9.824414


,valTotalEmpenhado,valLiquidado
txtDescricaoItemDespesa,,
PRODAM,15.912125,9.824414


,valTotalEmpenhado,valLiquidado
txtDescricaoPrograma,,
Modernização e descentralização da gestão e prestação de serviços,15.912125,9.824414


...

,valTotalEmpenhado,valLiquidado
txtDescricaoUnidade,,
Gabinete do Secretário,15.912125,9.824414


,valTotalEmpenhado,valLiquidado
txtGrupoDespesa,,
Outras Despesas Correntes,15.912125,9.824414


,valTotalEmpenhado,valLiquidado
txtModalidadeAplicacao,,
Aplicações Diretas,15.912125,9.824414


,valTotalEmpenhado,valLiquidado
txtRazaoSocial,,
EMPRESA DE TECNOLOGIA DA INFORMAÇÃO E COMUNICAÇÃO DO MUNICÍPIO DE SÃO PAULO - PRODAM-SP S/A,15.912125,9.824414


,txtObjetoContrato
7789,PRESTAÇÃO DE SERVIÇOS DE AGENTE DE RECREAÇÃO NO PROGRAMA RECREIO NAS FÉRIAS - JANEIRO/2017


# 10. Transferência de recursos PTRF

## 10.1 Geral

In [139]:
# mapear?

In [140]:
ptrf_filter = (
    (empenhos_df['txtDescricaoItemDespesa'].str.contains('Programa Transferência Recursos Financeiros', case=False))
)
ptrf = (empenhos_df[ptrf_filter])
empenhos_df.loc[ptrf_filter, 'cat_1'] = 'transferencias'
empenhos_df.loc[ptrf_filter, 'cat_2'] = 'ptrf'
groupSum(ptrf)

,valTotalEmpenhado,valLiquidado
txtCategoriaEconomica,,
Despesas Correntes,38.034370,37.074281
Despesas de Capital,4.844109,4.772288


,valTotalEmpenhado,valLiquidado
txtDescricaoElemento,,
Outros Serviços de Terceiros - Pessoa Jurídica,38.034370,37.074281
Equipamentos e Material Permanente,4.844109,4.772288


,valTotalEmpenhado,valLiquidado
txtDescricaoFonteRecurso,,
Tesouro Municipal,42.878479,41.84657


,valTotalEmpenhado,valLiquidado
txtDescricaoItemDespesa,,
Programa Transferência Recursos Financeiros - Assoc de Pais e Mestres,38.034370,37.074281
Programa Transferência Recursos Financeiros - PTRF - APM,4.844109,4.772288


,valTotalEmpenhado,valLiquidado
txtDescricaoPrograma,,
Melhoria da qualidade e ampliação do acesso à educação,42.861163,41.829254
Superação da extrema pobreza,0.017316,0.017316


...

,valTotalEmpenhado,valLiquidado
txtDescricaoUnidade,,
Diretoria Regional de Educação - Campo Limpo,6.001994,5.582000
Diretoria Regional de Educação - São Miguel,4.253045,4.150001
Diretoria Regional de Educação - Pirituba,4.297259,4.004503
Diretoria Regional de Educação - São Mateus,3.886844,3.847700
Diretoria Regional de Educação Capela do Socorro,3.087856,3.071113
Diretoria Regional de Educação - Guaianases,2.931090,2.899841
Diretoria Regional de Educação - Penha,2.909953,2.888076
Diretoria Regional de Educação - Jaçanã/Tremembé,2.881946,2.881946
Diretoria Regional de Educação - Freguesia/Brasilândia,2.737565,2.710469


,valTotalEmpenhado,valLiquidado
txtGrupoDespesa,,
Outras Despesas Correntes,38.034370,37.074281
Investimentos,4.844109,4.772288


,valTotalEmpenhado,valLiquidado
txtModalidadeAplicacao,,
Transferências a Instituições Privadas sem Fins Lucrativos,42.878479,41.84657


,valTotalEmpenhado,valLiquidado
txtRazaoSocial,,
APM EMEF PADRE JOSE PEGORARO,0.065233,0.065233
APM DA EMEFM ANTONIO A.VERISSIMO,0.064433,0.064433
APM DA EMEFM PROF. LINNEU PRESTES,0.063803,0.063803
APM DA EMEF FAGUNDES VARELLA,0.062930,0.062930
ASSOCIAÇÃO DE PAIS E MESTRES DA EMEF PADRE LEONEL FRANCA,0.096381,0.060962
A. P. M. DA EMEF CEU SAPOPEMBA,0.060248,0.060248
APM DO CEU EMEF MARIA APARECIDA DE SOUZA CAMPOS,0.059965,0.059965
APM DA EMEF FREI DAMIAO,0.059521,0.059521
APM DO CEU EMEF MANOEL VIEIRA DE QUEIROZ FILHO,0.059013,0.059013


,txtObjetoContrato
0,IT - 1º REPASSE PTRF PARA APM DA EMEI CAP. ALBERTO MENDES JUNIOR - DE ACORDO COM A LEI Nº 9394...
1,MP - 1º REPASSE PTRF PARA APM DO EMEI CLEMENCIA FERREIRA DA SILVA - DE ACORDO COM A LEI Nº 9394/...
2,"IT - 1º REPASSE PTRF PARA APM DO CEI JARDIM NAZARÉ - DE ACORDO COM A LEI Nº 9394//96, LEI MUNICI..."
3,"IT - 1º REPASSE PTRF PARA APM DA EMEI OVIDIO DECROLY - DE ACORDO COM A LEI Nº 9394//96, LEI MUN..."
4,MP - 1º REPASSE PTRF PARA APM DO EMEI PROFª MARIA DA LUZ SILVA DE OLIVEIRA - DE ACORDO COM A LE...
5,MP - 1º REPASSE PTRF PARA APM DO EMEI PROFª CELIA RIBEIRO LANDIM - DE ACORDO COM A LEI Nº 9394//...
6,IT - 1º REPASSE PTRF PARA APM DO CEI CONJ. HABITACIONAL TEXIMA - DE ACORDO COM A LEI Nº 9394//96...
7,MP - 1º REPASSE PTRF PARA APM DO EMEI DR. ANTONIO ROBERTO ALVES BRAGA - DE ACORDO COM A LEI Nº ...
8,"IT - 1º REPASSE PARA APM DA EMEF EUZEBIO ROCHA FILHO - DE ACORDO COM A LEI Nº 9394//96, LEI MUNI..."
9,"IT - 1º REPASSE PTRF PARA APM DO CEI VER. LIBERO ANCONA LOPES - DE ACORDO COM A LEI Nº 9394//96,..."


# Identificando categorias faltantes

In [141]:
faltantes = empenhos_df[empenhos_df['cat_1'].isnull()]
groupSum(faltantes)

,valTotalEmpenhado,valLiquidado
txtCategoriaEconomica,,
Despesas Correntes,76.063667,58.803747
Despesas de Capital,0.592846,0.132416


,valTotalEmpenhado,valLiquidado
txtDescricaoElemento,,
Outros Serviços de Terceiros -Pessoa Jurídica,59.324568,46.049149
Outros Serviços de Terceiros - Pessoa Física,8.599384,6.884361
Locação de Mão-de-Obra,2.827289,1.705485
Obrigações Tributárias e Contributivas,2.076094,1.321706
Auxílio-Alimentação,1.252159,1.248434
Material de Consumo,1.256918,1.106857
Despesas de Exercícios Anteriores,0.222071,0.211834
Equipamentos e Material Permanente,0.338062,0.132416
"Premiações Culturais, Artísticas, Científicas, Desportivas e Outras",0.107596,0.090112


,valTotalEmpenhado,valLiquidado
txtDescricaoFonteRecurso,,
Tesouro Municipal,76.651053,58.936163
Transferências Federais,0.005460,0.000000


,valTotalEmpenhado,valLiquidado
txtDescricaoItemDespesa,,
"Adiantamento - Incisos I, II e III, Art. 2º da Lei nº 10.513/88",12.433464,12.202874
"Supervisão, Acompanhamento e Suporte de Serviços Técnicos",14.722977,11.935234
Administração de Bens ou Negócios em Geral,6.073733,4.654509
Oficineiros,2.770065,2.548054
Jardinagem,2.610985,1.938594
Patrocínio a Projetos Culturais,1.800000,1.800000
Copa e Cozinha,2.612104,1.529440
SPTURIS,1.930917,1.448251
"Outros Serviços de Apoio Administrativo, Técnico e Operacional",1.616754,1.440490


,valTotalEmpenhado,valLiquidado
txtDescricaoPrograma,,
Melhoria da qualidade e ampliação do acesso à educação,57.707426,47.989619
Suporte Administrativo,10.675296,5.898537
Modernização e descentralização da gestão e prestação de serviços,4.557396,2.705148
Promoção da cidade como centro de eventos e destino turístico de refer,2.056456,1.448251
"Participação, transparência e controle social da administração pública",1.260375,0.546824
Direitos da pessoa com deficiência,0.399564,0.347784


...

,valTotalEmpenhado,valLiquidado
txtDescricaoUnidade,,
Gabinete do Secretário,42.772997,31.829956
Diretoria Regional de Educação - Penha,4.375267,3.578444
Diretoria Regional de Educação - Pirituba,2.974379,2.573592
Diretoria Regional de Educação - São Miguel,3.182867,2.493064
Diretoria Regional de Educação - Itaquera,4.552727,2.396378
Diretoria Regional de Educação - Santo Amaro,2.781061,2.353268
Diretoria Regional de Educação - Campo Limpo,2.470737,2.332096
Diretoria Regional de Educação - São Mateus,2.562849,2.253070
Diretoria Regional de Educação - Guaianases,2.292839,1.905254


,valTotalEmpenhado,valLiquidado
txtGrupoDespesa,,
Outras Despesas Correntes,76.063667,58.803747
Investimentos,0.592846,0.132416


,valTotalEmpenhado,valLiquidado
txtModalidadeAplicacao,,
Aplicações Diretas,76.536603,58.836391
Transferências a Instituições Privadas sem Fins Lucrativos,0.119910,0.099771


,valTotalEmpenhado,valLiquidado
txtRazaoSocial,,
INTEGRA SOLUCOES EM LOGISTICA E TRANSPORTE LTDA - EPP,6.073733,4.654509
FUNDACAO PARA O VESTIBULAR DA UNIVERSIDADE ESTADUAL PAULISTA JULIO DE MESQUITA FILHO VUNESP,3.917452,3.917452
FEDERAÇÃO DO DESPORTO ESCOLAR DO ESTADO DE SÃO PAULO,3.599508,3.466878
UNIAO SOCIAL CAMILIANA,2.731250,1.909764
SOCIEDADE DE CONCERTOS DE SÃO PAULO,2.269378,1.881276
POTENZA ENGENHARIA E CONSTRUÇÃO LTDA,2.461475,1.813287
EMPRESA DE CINEMA E AUDIOVISUAL DE SÃO PAULO S.A.,1.800000,1.800000
JOTABE SERVIÇOS TECNICOS ESPECIALIZADOS LTDA,3.295158,1.462790
SÃO PAULO TURISMO S/A.,2.056456,1.448251


,txtObjetoContrato
336,SP/SR AQUISIÇÃO DE 780 KG DE AÇUCAR NREFINADO AMORFO/CRISTALINO PACOTES COM 1 KG. CADA PARA USO ...
337,SP/SR AQUISIÇÃO DE 1.040 PACOTES DE CAFÉ COM 500 GR. CADA PARA USO NOS CEU'S JURISDIRECIONADOS A...
339,"SP/SR - REGISTRO DE PREÇOS PARA FORNECIMENTO PAPEL SULFITE A4 BRANCO COM CERTIFICADO AMBIENTAL, ..."
360,NaN
391,CONTRATAÇÃO DE AGENTES PARA RECREIO NAS FÉRIAS JUNHO/2017
407,SP/SR CONTRATAÇÃO DE SERVIÇOS DE AGUA E ESGOSTO REFERENTE O EXERCÍCIO DE 2017 PARA A UNIDADES DR...
437,SP/MP CONTRATAÇÃO DE TRADUTOR E INTÉRPRETE DE LIBRAS PERÍODO DE 18/09/2017 À 12/12/2017 - DRE/MP...
556,SP/SR AQUISIÇÃO DE 780 KG. DE AÇUCAR REFINADO AMORFO/CRISTALINO PACÓTES COM 500 GR. CADA PARA US...
562,SP/MP AQUISIÇÃO DE 400 GALÕES DE ÁGUA MINERAL SEM GÁS DE 20 LITROS PARA USO NO CEFOR
563,AQUISIÇÃO DE 400 KITS LANCHES PARA REUNIÃO A SER REALIZADA NO CEU VILA CURUÇA NO DIA 05/05/2017 ...


In [142]:
faltantes_det = empenhos_df[(
#     empenhos_df['cat_1'].isnull()
#     &
    (empenhos_df['txtDescricaoItemDespesa'].str.contains('Eventos', case=False))
)]
groupSum(faltantes_det)

,valTotalEmpenhado,valLiquidado
txtCategoriaEconomica,,
Despesas Correntes,8.122321,6.599255


,valTotalEmpenhado,valLiquidado
txtDescricaoElemento,,
Outros Serviços de Terceiros -Pessoa Jurídica,8.122321,6.599255


,valTotalEmpenhado,valLiquidado
txtDescricaoFonteRecurso,,
Tesouro Municipal,8.122321,6.599255


,valTotalEmpenhado,valLiquidado
txtDescricaoItemDespesa,,
"Eventos Esportivos, Culturais e Turísticos",7.310114,5.812939
"Eventos Esportivos, Culturais e Turísticos - Cooperativas",0.708000,0.684800
Eventos Esportivos Culturais e Artísticos,0.071757,0.069066
Refeições para Eventos em Educação,0.032450,0.032450


,valTotalEmpenhado,valLiquidado
txtDescricaoPrograma,,
Melhoria da qualidade e ampliação do acesso à educação,7.933431,6.485365
Modernização e descentralização da gestão e prestação de serviços,0.152000,0.077000
Suporte Administrativo,0.036890,0.036890


...

,valTotalEmpenhado,valLiquidado
txtDescricaoUnidade,,
Gabinete do Secretário,7.915108,6.394734
Diretoria Regional de Educação - Penha,0.054418,0.054418
Diretoria Regional de Educação - Butantã,0.029795,0.029795
Diretoria Regional de Educação - Guaianases,0.029005,0.029005
Diretoria Regional de Educação - São Mateus,0.021710,0.021710
Diretoria Regional de Educação - Jaçanã/Tremembé,0.017338,0.014678
Diretoria Regional de Educação - Campo Limpo,0.014595,0.014595
Diretoria Regional de Educação - Itaquera,0.014192,0.014160
Diretoria Regional de Educação - São Miguel,0.012300,0.012300


,valTotalEmpenhado,valLiquidado
txtGrupoDespesa,,
Outras Despesas Correntes,8.122321,6.599255


,valTotalEmpenhado,valLiquidado
txtModalidadeAplicacao,,
Aplicações Diretas,8.122321,6.599255


,valTotalEmpenhado,valLiquidado
txtRazaoSocial,,
TOKA EVENTOS ARTISTICOS EIRELI,0.702000,0.636000
COOPERATIVA PAULISTA DE TEATRO,0.564000,0.548800
CONFEDERAÇÃO BRASILEIRA DE ARTES MARCIAIS CHINESAS - KUNG FU,0.630115,0.381328
BAIOQUE PRODUCOES ARTISTICAS LTDA - ME,0.350000,0.350000
FEDERAÇÃO DO DESPORTO ESCOLAR DO ESTADO DE SÃO PAULO,0.330915,0.279311
BOLZAN E ASCHENBACH EVENTOS LTDA,0.276000,0.276000
FEDERAÇÃO PAULISTA DE KARATE,0.258095,0.208461
NAJARA CINE VIDEO E EVENTOS LTDA,0.603800,0.200200
FEDERAÇÃO PAULISTA DE ARTES MARCIAIS INTERESTILOS,0.223940,0.194742


,txtObjetoContrato
682,"AUTORIZO a Contratação de Arbitragem conforme Memorando Inicial Nº SEI 3884008, cuja detentora é..."
4929,NaN
4932,Locação de Brinquedos Infláveis - Recreio nas Férias - Edição Julho/2017.
4933,Apresentações do espetáculo: “Zuando Som Apresenta Faz de Conta”
4934,"Contratação da empresa ""Super Produções Geniais Ltda. ME"", com vistas à realização de 08 (oito) ..."
4935,Apresentações do Espetáculo “QUATRO FACES DO HERÓI”
4936,Apresentações do Espetáculo “CONTOS DE GRIMM: PRINCESAS E BRUXAS”
4937,"Apresentações do Espetáculo ""Números de Circo - Um Espetáculo para Crianças"""
4938,Apresentações do Espetáculo “AVESSO DO AVESSO”
4939,"Apresentações do Espetáculo ""ATÉ AS PRINCESAS SOLTAM PUM"""


In [143]:
# CEU - outros
ceu_filter = (
    empenhos_df['cat_1'].isnull()
    &
    (empenhos_df['txtDescricaoProjetoAtividade'].str.contains('Operação e Manutenção dos Centros Educacionais Unificados', case=False))
)
ceu = (empenhos_df[ceu_filter])
empenhos_df.loc[ceu_filter, 'cat_1'] = 'funcionamento_unidades'
empenhos_df.loc[ceu_filter, 'cat_2'] = 'ceu_outros'
# groupSum(ceu)

In [144]:
# suporte adm - outros
suporte_administrativo_filter = (
    empenhos_df['cat_1'].isnull()
    &
    (empenhos_df['txtDescricaoPrograma'].str.contains('Suporte Administrativo', case=False))
)
suporte_administrativo = (empenhos_df[suporte_administrativo_filter])
empenhos_df.loc[suporte_administrativo_filter, 'cat_1'] = 'suporte_administrativo'
empenhos_df.loc[suporte_administrativo_filter, 'cat_2'] = 'suporte_administrativo_outros'
# groupSum(suporte_administrativo)

# Export

In [145]:
# clean errors, columns with cat_1 but no cat_2
errors_filter = ((empenhos_df['cat_2'].isnull() | (empenhos_df['cat_2'] == '')) & ~empenhos_df['cat_1'].isnull())
print('erros:', len(empenhos_df[errors_filter]))
# empenhos_df.loc[errors_filter, 'cat_1'] = ''
empenhos_df.loc[errors_filter, 'cat_2'] = 'outros'
empenhos_df[errors_filter]['txtDescricaoItemDespesa'].unique()

erros: 283


array(['Convênio - Creche', 'Toner e Cartuchos de Tinta',
       'Manutenção de Software', 'Certificado Digital',
       'Guarda e Gerenciamento de Documentos',
       'Supervisão, Acompanhamento e Suporte de Serviços Técnicos',
       'Pesquisa em Geral',
       'Outros Serviços de Apoio Administrativo, Técnico e Operacional',
       'Aquisição de Softwares - Intangível', 'Hardware', 'Switch ',
       'Outros Aparelhos e Equipamentos de Comunicação',
       'Outras Máquinas e Equipamentos Energéticos ',
       'Convênios - Portadores de Deficiência',
       'Outras Despesas Fixas - Pessoal Civil',
       'Convênios - Alfabetização, Educação e Outros'], dtype=object)

In [146]:
# # todos outros restantes, em uma categoria "outros"
# # clean errors, columns with cat_1 but no cat_2
# errors_filter = ((empenhos_df['cat_2'].isnull() | (empenhos_df['cat_2'] == '')) & empenhos_df['cat_1'].isnull())
# print('erros:', len(empenhos_df[errors_filter]))
# empenhos_df.loc[errors_filter, 'cat_1'] = 'outros'
# empenhos_df.loc[errors_filter, 'cat_2'] = 'outros'
# empenhos_df[errors_filter]['txtDescricaoItemDespesa'].unique()

In [147]:
# coverage
coverage = empenhos_df[~empenhos_df['cat_1'].isnull() & ~empenhos_df['cat_2'].isnull()]['valLiquidado'].sum() / empenhos_df['valLiquidado'].sum()
print(f"cobertura do orçamento: {round(coverage * 100, 2)}%")
print(f"valor faltante {round(empenhos_df[empenhos_df['cat_1'].isnull() & empenhos_df['cat_2'].isnull()]['valLiquidado'].sum() / 1000000, 2)} mi")

cobertura do orçamento: 99.5%
valor faltante 50.82 mi


In [148]:
out = (empenhos_df[['valLiquidado', 'cat_1', 'cat_2']]         
    .groupby(['cat_1', 'cat_2'])
    .sum().apply(lambda x: x / 1000000)
#     .sort_values('valLiquidado', ascending=False)
)
display(out)

valLiquidado
cat_1                  cat_2                                      
alimentacao            insumos                          194.498042
                       leve_leite                        50.826933
                       pnae                              19.871110
                       prep_dist                         52.905599
                       refeicoes                        328.161845
funcionamento_unidades agua_esgoto                       93.020801
                       aluguel_imoveis                   17.577437
                       ceu_outros                        25.619076
                       eletricidade                      44.973127
                       eventos                            6.497739
                       limpeza                          406.884942
                       manutencao                        17.065857
                       mobiliario                         0.011702
                       obras_reformas                    58.263296
                       telefone                           5.298665
                       vigilancia                       181.857045
parcerias              creches                         1954.064406
                       ed_especial                       36.483657
                       eja                               10.687166
                       outros                             3.005698
pessoal                docentes                        5872.021815
                       estagios                          33.448145
                       gestores                         343.097216
                       outros                             2.179419
suporte_administrativo suporte_administrativo_outros      5.898537
ti                     internet                          11.271799
                       manutencao                         0.983508
                       manutencao_equip_ti                5.937811
                       outros                            12.879808
                       prodam                             9.824414
transferencias         ptrf                              41.846570
transporte             teg                              194.491393
                       veiculos                          15.920269
uniforme_materiais     materiais                         26.113732
                       uniforme                         101.825344

In [149]:
# format data for treemap visualization
index_1 = list(out.index.get_level_values(0).unique())
index_2 = list(out.index.get_level_values(1).unique())
formatted_out = {'name': 'orc', 'children': []}
for i in index_1:
    formatted_out['children'].append({'name': i, 'children': []})
for index, row in out.iterrows():
    cat_1 = index[0]
    cat_2 = index[1]
    # display as millions?
    value = round(row['valLiquidado'] * 1000000, 2)
    for child_index, child in enumerate(formatted_out['children']):
        if child['name'] == cat_1:
            formatted_out['children'][child_index]['children'].append({'name': cat_2, 'size': value})
pprint.pprint(formatted_out)

{'children': [{'children': [{'name': 'insumos', 'size': 194498042.16},
                            {'name': 'leve_leite', 'size': 50826933.02},
                            {'name': 'pnae', 'size': 19871110.0},
                            {'name': 'prep_dist', 'size': 52905599.38},
                            {'name': 'refeicoes', 'size': 328161844.67}],
               'name': 'alimentacao'},
              {'children': [{'name': 'agua_esgoto', 'size': 93020801.3},
                            {'name': 'aluguel_imoveis', 'size': 17577437.31},
                            {'name': 'ceu_outros', 'size': 25619075.73},
                            {'name': 'eletricidade', 'size': 44973126.74},
                            {'name': 'eventos', 'size': 6497739.28},
                            {'name': 'limpeza', 'size': 406884941.54},
                            {'name': 'manutencao', 'size': 17065857.04},
                            {'name': 'mobiliario', 'size': 11702.0},
                        

In [150]:
with open('budget.json', 'w') as f:
    f.write(json.dumps(formatted_out))